# Data analysis for temporal predictability

In [1]:
### Necessary packages ###
import mne
import numpy as np
from pathlib import Path
from autoreject import Ransac
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import seaborn as sns
import pandas as pd
%matplotlib qt
import matplotlib.ticker as ticker 
from IPython.display import display
# DATA ANALYSIS PART
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
import pingouin as pg

## Preprocessing for data from 2023 ##

### Checking triggers and data ###

In [2]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data4\\BB")

# conditions we are interested in as a list
conds_of_interest = ["AR"]
conds_of_interest2 = ["US","AS","UR","AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5
counter = 0

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_BB_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        display(raw)
        
        raw.plot()
        raw.set_montage('standard_1020')
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        def check_event_triggers(events):
            ignore_codes = set(range(1000, 1101))
            trigger_counts = {2: 0, 4: 0, 6: 0, 8: 0, 10: 0}
            
            # Process each event
            for i, event in enumerate(events):
                # Skip ignored codes and the code 99999
                if event[2] in ignore_codes or event[2] == 99999:
                    continue

                # Count valid triggers
                if event[2] in trigger_counts:
                    trigger_counts[event[2]] += 1

            return trigger_counts


        # Now let's call the function and print the results
        trigger_counts = check_event_triggers(events)
        for trigger, count in trigger_counts.items():
            print(f"Trigger {trigger} count: {count}")
    
        
    
        
        
    


Extracting parameters from C:\Users\mtu10\OneDrive\Desktop\EEG_code\Data4\BB\AR_BB_01.vhdr...
Setting channel info structure...
Reading 0 ... 170759  =      0.000 ...   170.759 secs...


C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\2738457531.py:39: RuntimeWarning: No coordinate information found for channels ['VEOG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(file,preload=True)
C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\2738457531.py:39: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(file,preload=True)
C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\2738457531.py:39: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['VEOG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(file,preload=True)


Measurement date,"July 19, 2023 16:08:19 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,34 points
Good channels,31 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.01 Hz
Lowpass,120.00 Hz


Using matplotlib as 2D backend.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.03 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.03
- Lower transition bandwidth: 0.03 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 110001 samples (110.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


...

C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\2738457531.py:39: RuntimeWarning: No coordinate information found for channels ['VEOG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(file,preload=True)
C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\2738457531.py:39: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(file,preload=True)
C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\2738457531.py:39: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['VEOG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(file,preload=True)


Measurement date,"July 19, 2023 16:28:55 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,34 points
Good channels,31 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.01 Hz
Lowpass,120.00 Hz


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.03 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.03
- Lower transition bandwidth: 0.03 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 110001 samples (110.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Used Annotations descriptions: ['New Segment/', 'Response/R  7', 'Response/R 13', 'Response/R 15', 'Stimulus/S  2', 'Stimulus/S  4', 'Stimulus/S  6']
Trigger 2 count: 114
Trigger 4 count: 114
Trigger 6 count: 76
Trigger 8 count: 0
Trigger 10 count: 0


#### Preprocessing for BA;BC;BD --> BB is excluded

### Participant BC

In [3]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data4\\BC")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","UR"]
conds_of_interest2 = ["AR"]
# tmin and tmax of epoch
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_BC_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID 
    block_number = file_parts[2]  # Block number

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        raw.plot()
        raw.set_montage('standard_1020')
        ## drop noisy channels
        raw.drop_channels(['P4'])
        raw.drop_channels(['C4'])
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs4')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        
        
        # filter data
        raw.filter(0.03,25)
        raw.drop_channels(['P4'])
        raw.drop_channels(['C4'])
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)   
        
        
        
    


Extracting parameters from C:\Users\mtu10\OneDrive\Desktop\EEG_code\Data4\BC\AR_BC_01.vhdr...
Setting channel info structure...
Reading 0 ... 165339  =      0.000 ...   165.339 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.03 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.03
- Lower transition bandwidth: 0.03 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 110001 samples (110.001 s)



C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\498812730.py:128: RuntimeWarning: No coordinate information found for channels ['VEOG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(file,preload=True)
C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\498812730.py:128: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(file,preload=True)
C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\498812730.py:128: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['VEOG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(file,preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['New Segment/', 'Response/R  7', 'Response/R 15', 'Stimulus/S  2', 'Stimulus/S  4', 'Stimulus/S  6']
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 304 events and 601 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    2.5s remaining:   23.2s
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:    2.5s remaining:    7.7s
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed:    2.6s remaining:    3.9s
[Parallel(n_jobs=20)]: Done  11 out of  20 | elapsed:    2.6s remaining:    2.1s
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed:    2.7s remaining:    1.1s
[Parallel(n_jobs=20)]: Done  17 out of  20 | elapsed:    2.7s remaining:    0.4s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    2.8s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.9s remaining:    8.5s
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:    0.9s remaining:    3.0s
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed:    0.9s remaining:    1.5s
[Parallel(n_jobs=20)]: Done  11 out of  20 | e

[Done]
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 96.0 mm
Computing interpolation matrix from 28 sensor positions
Interpolating 1 sensors
Oz
Fitting ICA to data using 29 channels (please be patient, this may take a while)
Selecting by number: 28 components
Fitting ICA took 4.5s.
Using EOG channels: Fp1, Fp2
Applying ICA to Epochs instance
    Transforming to ICA space (28 components)
    Zeroing out 1 ICA component
    Projecting back using 29 PCA components
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Applying baseline correction (mode: mean)
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejecting  epoch based on EEG : ['Fp1']
    Rejectin

...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['New Segment/', 'Response/R  7', 'Response/R 13', 'Response/R 15', 'Stimulus/S  2', 'Stimulus/S  4', 'Stimulus/S  8', 'Stimulus/S 10']
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 304 events and 601 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Batch computation too fast (0.04029107093811035s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  11 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  17 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.8s remaining:    8.4s
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:    0.9s remaining:    3.0s
[Parallel(n_jobs=20)]: Done   8

[Done]
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 96.0 mm
Computing interpolation matrix from 28 sensor positions
Interpolating 1 sensors
FC6
Fitting ICA to data using 29 channels (please be patient, this may take a while)
Selecting by number: 28 components
Fitting ICA took 4.4s.
Using EOG channels: Fp1, Fp2
Applying ICA to Epochs instance
    Transforming to ICA space (28 components)
    Zeroing out 1 ICA component
    Projecting back using 29 PCA components
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Applying baseline correction (mode: mean)
    Rejecting  epoch based on EEG : ['Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based 

Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none


#### Participant BA

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data4\\BA")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","UR","AR"]
#conds_of_interest2 = ["UR"]
# tmin and tmax of epoch
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_BA_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID 
    block_number = file_parts[2]  # Block number

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        raw.plot()
        raw.set_montage('standard_1020')
    
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()

        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.75)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs4')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 
    
        
    


### Participant BD

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data4\\BD")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","UR","AR"]
#conds_of_interest2 = ["AR"]
# tmin and tmax of epoch
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_BD_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        raw.plot()
        raw.set_montage('standard_1020')

        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.65)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs4')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 
    
        
    


### Read epochs file and save it as evoked ####

In [2]:
def load_and_combine_epochs_for_condition(part_directory, condition):
    epochs_list = []
    for block_file in part_directory.glob(f"{condition}_{part_directory.stem}_*-epo.fif"):
        epochs = mne.read_epochs(block_file, preload=True)
        # Check if this is participant and adjust channels
        if part_directory.stem == "BC":
            # Drop specific channels if they are presentV
            channels_to_drop = [ch for ch in ['C4', 'P4'] if ch in epochs.ch_names]
            epochs.drop_channels(channels_to_drop)
        epochs_list.append(epochs)
    combined_epochs = mne.concatenate_epochs(epochs_list)
    return combined_epochs

conds_of_interest = ["US","AS","UR","AR"]

# Main processing loop
all_parts23 = {}
for part in Path("saved_epochs4").iterdir():
    evokeds = []
    for cond in conds_of_interest:
        # Load and combine epochs for each condition across blocks
        combined_epochs = load_and_combine_epochs_for_condition(part, cond)
        
        if "A" in cond:
            lat_cond = [f"{cond}/lat_am"]
        elif "U" in cond:
            lat_cond = [f"{cond}/lat_unam_r", f"{cond}/lat_unam_l"]

        evoked = combined_epochs[lat_cond].average()
        evoked.comment = cond  # Simplify condition name directly here
        evokeds.append(evoked)

    all_parts23[part.stem] = evokeds
    
### delete additional info in conditions
def simplify_condition(comment):
    for cond in conds_of_interest:
        if cond in comment:
            return cond
    return None

def simplify_evoked_conditions(participant_evokeds):
    simplified_evokeds = []
    for evoked in participant_evokeds:
        simplified_comment = simplify_condition(evoked.comment)
        if simplified_comment:
            evoked.comment = simplified_comment
            simplified_evokeds.append(evoked)
    return simplified_evokeds
for participant in list(all_parts23):
    all_parts23[participant] = simplify_evoked_conditions(all_parts23[participant])

Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BA\US_BA_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BA\US_BA_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
604 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BA\AS_BA_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\4233796953.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


225 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BA\AS_BA_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
464 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BA\UR_BA_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
201 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BA\UR_BA_02-epo.fif ...
    Found the data of interest:


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\4233796953.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
505 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BA\AR_BA_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available


...

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\4233796953.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


304 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BD\UR_BD_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
276 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
580 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BD\AR_BD_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\4233796953.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


304 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs4\BD\AR_BD_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
287 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
591 matching events found
Applying baseline correction (mode: mean)


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\4233796953.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


### Processing of dataset 2023 ###

In [17]:
all_parts23

{'BA': [<Evoked | 'US' (average, N=151), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AS' (average, N=145), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'UR' (average, N=130), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AR' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>],
 'BC': [<Evoked | 'US' (average, N=89), -0.1 – 0.5 s, baseline -0.1 – 0 s, 29 ch, ~178 kB>,
  <Evoked | 'AS' (average, N=96), -0.1 – 0.5 s, baseline -0.1 – 0 s, 29 ch, ~178 kB>,
  <Evoked | 'UR' (average, N=150), -0.1 – 0.5 s, baseline -0.1 – 0 s, 29 ch, ~178 kB>,
  <Evoked | 'AR' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 29 ch, ~178 kB>],
 'BD': [<Evoked | 'US' (average, N=151), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AS' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'UR' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AR' (average

### Plot individual evoked data for participants in dataset 2023 ###

In [18]:


def plot_evokeds_for_participant(participant, evokeds, colors):
    # Define styles for the lines
    styles = {
        'US': {'color': 'red', 'linestyle': '-'},   # Bold red for Unambiguous-Same
        'UR': {'color': 'red', 'linestyle': '--'},  # Dashed red for Unambiguous-Random
        'AS': {'color': 'blue', 'linestyle': '-'},  # Bold blue for Ambiguous-Same
        'AR': {'color': 'blue', 'linestyle': '--'}  # Dashed blue for Ambiguous-Random
    }
    
    # Determine the common channels among all evokeds for the participant
    common_channels = set(evokeds[0].ch_names)
    for evoked in evokeds[1:]:
        common_channels.intersection_update(evoked.ch_names)
    
    # Pick only the common channels for plotting
    evokeds_common_channels = []
    for ev in evokeds:
        ev.pick_channels(list(common_channels))
        evokeds_common_channels.append(ev)

    # Adjust colors and styles based on the evoked.comment
    adjusted_styles = {ev.comment: styles[ev.comment] for ev in evokeds_common_channels if ev.comment in styles}
    adjusted_colors = {ev.comment: colors[ev.comment] for ev in evokeds_common_channels if ev.comment in colors}

    # Plotting using adjusted styles and colors
    mne.viz.plot_compare_evokeds(
        {ev.comment: ev for ev in evokeds_common_channels},
        title=f"Participant '{participant}' ERPs",
        picks="Cz",
        colors=adjusted_colors,
        styles=adjusted_styles,
        ci=True,
        show=True,
        truncate_yaxis=False,
        truncate_xaxis=False
    )
colors = {'US': 'red', 'AS': 'blue', 'UR': 'red', 'AR': 'blue'}  

for participant, evokeds in all_parts.items():
    if evokeds:  # Only plot if there are evoked data present
        plot_evokeds_for_participant(participant, evokeds, colors)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\519064909.py:20: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\519064909.py:20: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\519064909.py:20: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting cha

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\519064909.py:20: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\519064909.py:20: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\519064909.py:20: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting cha

### Grand mean of dataset 2023 ###

In [19]:


combined_evokeds_by_cond = {cond: [] for cond in conds_of_interest}
participant_count_by_cond = {}  # Dictionary to hold participant counts for each condition

for participant, evokeds in all_parts.items():
    for evoked in evokeds:
        cond = simplify_condition(evoked.comment)
        if cond:
            combined_evokeds_by_cond[cond].append(evoked)
            if cond not in participant_count_by_cond:
                participant_count_by_cond[cond] = set()
            participant_count_by_cond[cond].add(participant)

# Adjusting each evoked data to only include common channels
all_channels = set.intersection(*(set(evoked.ch_names) for evokeds in combined_evokeds_by_cond.values() for evoked in evokeds))
for cond, evokeds in combined_evokeds_by_cond.items():
    for evoked in evokeds:
        evoked.pick_channels(list(all_channels))  # Updated method to use pick_channels

grand_averages_all = []
legend_info = {}
for cond, evokeds in combined_evokeds_by_cond.items():
    if evokeds:
        grand_avg = mne.grand_average(evokeds)
        grand_avg.comment = cond
        grand_averages_all.append(grand_avg)
        # Prepare legend info with participant count
        legend_info[cond] = f"{cond} (n={len(participant_count_by_cond[cond])})"

# Dictionary for plot styling
colors = {
    'US': 'red',
    'UR': 'red',
    'AS': 'blue',
    'AR': 'blue'
}
styles = {
    'UR': {'linestyle': '--'},
    'AR': {'linestyle': '--'}
}

# Update styles and colors to use the legend_info keys
updated_colors = {legend_info[cond]: colors[cond] for cond in colors if cond in legend_info}
updated_styles = {legend_info[cond]: styles[cond] for cond in styles if cond in legend_info}

# Plotting
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(
    {legend_info[cond]: grand_averages_all[i] for i, cond in enumerate(conds_of_interest) if cond in legend_info},
    picks="Cz",
    ci=0.95,
    colors=updated_colors,
    styles=updated_styles,
    title="Grand Average Evoked Responses by Condition",
    axes=ax
)

# Customizing the legend manually
handles, labels = ax.get_legend_handles_labels()
new_labels = [legend_info[label.split()[0]] for label in labels]  # Remapping labels to include participant counts
ax.legend(handles, new_labels)
plt.show()


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Identifying common channels ...
Identify

C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\1319273522.py:17: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  evoked.pick_channels(list(all_channels))  # Updated method to use pick_channels
C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\1319273522.py:17: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  evoked.pick_channels(list(all_channels))  # Updated method to use pick_channels
C:\Users\mtu10\AppData\Local\Temp\ipykernel_14032\1319273522.py:17: FutureWarning: The default for pick_channels will change from ordered=False to ordered

### Grand mean with confidence interval ###

In [20]:

combined_evokeds_by_cond = {cond: [] for cond in conds_of_interest}
participant_count_by_cond = {}  # Dictionary to hold participant counts for each condition

for participant, evokeds in all_parts.items():
    for evoked in evokeds:
        cond = simplify_condition(evoked.comment)
        if cond:
            combined_evokeds_by_cond[cond].append(evoked)
            if cond not in participant_count_by_cond:
                participant_count_by_cond[cond] = set()
            participant_count_by_cond[cond].add(participant)

# Dictionary for plot styling
colors = {
    'US': 'red',
    'UR': 'red',
    'AS': 'blue',
    'AR': 'blue'
}
styles = {
    'UR': {'linestyle': '--'},
    'AR': {'linestyle': '--'}
}

# Prepare the evoked data with participant counts for plotting
plot_evokeds = {}
for cond in combined_evokeds_by_cond:
    if combined_evokeds_by_cond[cond]:
        plot_evokeds[f"{cond} (n={len(participant_count_by_cond[cond])})"] = combined_evokeds_by_cond[cond]

# Update colors and styles to use the condition names with participant counts
updated_colors = {f"{cond} (n={len(participant_count_by_cond[cond])})": colors[cond] for cond in colors}
updated_styles = {f"{cond} (n={len(participant_count_by_cond[cond])})": styles[cond] for cond in styles if cond in styles}

# Plotting with CIs
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(
    plot_evokeds,
    picks="Cz",
    ci=0.95,
    colors=updated_colors,
    styles=updated_styles,
    title="Grand Average Evoked Responses by Condition",
    axes=ax
)

plt.show()


### Analysis of dataset 2022 ###

### Preprocessing ###

### Checking triggers ###

In [ ]:

# get data path to specific participant 
### AM is excluded
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AM")

# conditions we are interested in as a list
conds_of_interest = ["UR"]
conds_of_interest2 = ["US","AS","UR","AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5
counter = 0

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AM_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        display(raw)
        
        raw.plot()
        raw.set_montage('standard_1020')
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        def check_event_triggers(events):
            ignore_codes = set(range(1000, 1101))
            trigger_counts = {2: 0, 4: 0, 6: 0, 8: 0, 10: 0}
            
            # Process each event
            for i, event in enumerate(events):
                # Skip ignored codes and the code 99999
                if event[2] in ignore_codes or event[2] == 99999:
                    continue

                # Count valid triggers
                if event[2] in trigger_counts:
                    trigger_counts[event[2]] += 1

            return trigger_counts


        # Now let's call the function and print the results
        trigger_counts = check_event_triggers(events)
        for trigger, count in trigger_counts.items():
            print(f"Trigger {trigger} count: {count}")
    
        
    
        
        
    


## Preprocess the dataset 2022

##### Participant AT ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AT")

# conditions we are interested in as a list
conds_of_interest = ["US"]
conds_of_interest2 = [ "AR", "UR", "AS"]
# tmin and tmax of epoch
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AT_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number
    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        
        # Drop noisy channels
        raw.drop_channels(['FC6', 'CP2'])
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        
        raw.pick_types(eeg=True)
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        # raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.6)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.6)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)   
        
        
        
    


#### Participant AJ ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AJ")

# conditions we are interested in as a list
conds_of_interest = ["AS","UR","AR"]
conds_of_interest2 = ["US"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AJ_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        # epochs.set_eeg_reference(ref_channels='average', projection=False) ###############
        epochs.set_eeg_reference(['P7','P8'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        
        raw.drop_channels(['FC6'])
        # raw.drop_channels(['CP2'])
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        # epochs.set_eeg_reference(ref_channels='average', projection=False) ###############
        epochs.set_eeg_reference(['P7','P8'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
        
        
    


#### Participant AK ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AK")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","UR","AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AK_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number 

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        raw.plot()
        raw.set_montage('standard_1020')
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        # rereferenced because TP10 is noisy
        epochs.set_eeg_reference(['P7','P8'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
        
        
        
    


#### Participant AL ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AL")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","UR"]
conds_of_interest2 = ["AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AL_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.8)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
    if file_cond in conds_of_interest2 and block_number == '01': ## AR Block 2 is recorded wrongly.
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')

        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.8)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
        
        
    


#### Participant AM ####

In [ ]:

"""-> AM is excluded"""
# # get data path to specific participant
# participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AM")

# # conditions we are interested in as a list
# conds_of_interest = ["US","AS","UR","AR"]
# # tmin and tmax of epochs
# tmin = -0.1
# tmax = 0.5

# # Participant name
# participant_id = participant_path.stem

# for file in participant_path.glob("*_AM_*.vhdr"):
#     # Extract the condition and block from the file name 
#     file_parts = file.stem.split('_')
#     file_cond = file_parts[0]  # Condition (e.g., "AR")
#     participant_id = file_parts[1]  # Participant ID
#     block_number = file_parts[2]  # Block number (e.g., "01")

#     if file_cond in conds_of_interest:
#         # event triggers as dict used file cond info
#         event_id_am = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_am": 6}
#         event_id_unam = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_unam_l": 8,
#                     f"{file_cond}/lat_unam_r": 10}
#         # get event_id corresponding with condition
#         if "A" in file_cond:
#             event_id = event_id_am
#         elif "U" in file_cond:
#             event_id = event_id_unam
        
                    
#        # load raw data
#         raw = mne.io.read_raw_brainvision(file,preload=True)
#         # only pick eeg channels
#         raw.pick("eeg")
        
#         # raw.plot()
#         raw.set_montage('standard_1020')
#         ####
#         # orig_events, _ = mne.events_from_annotations(raw)
#         # events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
#         # print(events)
        
        
#         # filter data
#         raw.filter(0.03,25)
#         # raw.plot_psd(fmax=40)
#         raw.pick(["eeg"]).load_data()
#         # raw.info
#         # raw.plot_sensors(show_names=True)
#         # fig = raw.plot_sensors("3d")
#         # fig
        
#         # get triggers
#         orig_events, _ = mne.events_from_annotations(raw)
#         events = orig_events
#         events.shape
#         epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

#         picks = mne.pick_types(epochs.info, meg=False, eeg=True,
#         stim=False, eog=False,
#         include=[], exclude=[])
#         ransac = Ransac(picks=picks, n_jobs=-1)
#         epochs = ransac.fit_transform(epochs)
#         print('\n'.join(ransac.bad_chs_))
        
#         # epochs.plot()
#         # compute an ICA to remove eye blink components from signal
#         ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
#         ica.fit(epochs)
#         ica.exclude = []
#         eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
#         ica.exclude = eog_idx
#         # ica.plot_scores(scores)
#         ica.plot_components()
#         ica.plot_sources(epochs)
#         epochs_new = epochs.copy()
#         ica.apply(epochs_new)
#         epochs = epochs_new
#         # epochs.plot()
        
#         # set offline reference at mastoid electrodes
#         epochs.set_eeg_reference(['TP9','TP10'], projection=False)
#         # apply baseline in ms
#         epochs.apply_baseline((-0.1,0))
#         # reject epochs that are above a certain threshold
#         # value needs to be in volts
#         epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
#         # epochs.plot()
#         # epochs.plot_drop_log()
#         # save epoch objects and events array
#         epoch_path = Path('saved_epochs3')
#         folder_path = Path(epoch_path,participant_id)
#         # Check if saved_epochs folder already exists
#         if not epoch_path.exists():
#             # Create the folder
#             epoch_path.mkdir(parents=True)
#         # Check if participant folder already exists
#         if not folder_path.exists():
#             # Create the folder
#             folder_path.mkdir(parents=True)
#         # Construct the filename with condition, participant ID, and block number
#         epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
#         annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
#         # Save the epochs and annotations
#         epochs.save(folder_path / epochs_filename, overwrite=True)
#         np.save(folder_path / annotations_filename, events)    
        
        
        
    


#### Participant AN ####

In [ ]:
"""--> AN is excluded"""


# # get data path to specific participant
# participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AN")

# # conditions we are interested in as a list
# conds_of_interest = ["US","AS","UR","AR"]

# # tmin and tmax of epochs
# tmin = -0.1
# tmax = 0.5

# # Participant name
# participant_id = participant_path.stem

# for file in participant_path.glob("*_AN_*.vhdr"):
#     # Extract the condition and block from the file name 
#     file_parts = file.stem.split('_')
#     file_cond = file_parts[0]  # Condition (e.g., "AR")
#     participant_id = file_parts[1]  # Participant ID (e.g., "AB")
#     block_number = file_parts[2]  # Block number (e.g., "01")

#     if file_cond in conds_of_interest:
#         # event triggers as dict used file cond info
#         event_id_am = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_am": 6}
#         event_id_unam = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_unam_l": 8,
#                     f"{file_cond}/lat_unam_r": 10}
#         # get event_id corresponding with condition
#         if "A" in file_cond:
#             event_id = event_id_am
#         elif "U" in file_cond:
#             event_id = event_id_unam
        
                    
#        # load raw data
#         raw = mne.io.read_raw_brainvision(file,preload=True)
#         # only pick eeg channels
#         raw.pick("eeg")
        
#         # raw.plot()
#         raw.set_montage('standard_1020')
#         ####
#         # orig_events, _ = mne.events_from_annotations(raw)
#         # events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
#         # print(events)
        
        
#         # filter data
#         raw.filter(0.03,25)
#         # raw.plot_psd(fmax=40)
#         raw.pick(["eeg"]).load_data()
#         # raw.info
#         # raw.plot_sensors(show_names=True)
#         # fig = raw.plot_sensors("3d")
#         # fig
        
#         # get triggers
#         orig_events, _ = mne.events_from_annotations(raw)
#         events = orig_events
#         events.shape
#         epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

#         picks = mne.pick_types(epochs.info, meg=False, eeg=True,
#         stim=False, eog=False,
#         include=[], exclude=[])
#         ransac = Ransac(picks=picks, n_jobs=-1)
#         epochs = ransac.fit_transform(epochs)
#         print('\n'.join(ransac.bad_chs_))
        
#         # epochs.plot()
#         # compute an ICA to remove eye blink components from signal
#         ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
#         ica.fit(epochs)
#         ica.exclude = []
#         eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.6)
#         ica.exclude = eog_idx
#         # ica.plot_scores(scores)
#         # ica.plot_components()
#         # ica.plot_sources(epochs)
#         epochs_new = epochs.copy()
#         ica.apply(epochs_new)
#         epochs = epochs_new
#         # epochs.plot()
        
#         # set offline reference at mastoid electrodes
#         epochs.set_eeg_reference(['TP9','TP10'], projection=False)
#         # apply baseline in ms
#         epochs.apply_baseline((-0.1,0))
#         # reject epochs that are above a certain threshold
#         # value needs to be in volts
#         epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
#         # epochs.plot()
#         # epochs.plot_drop_log()
#         # save epoch objects and events array
#         epoch_path = Path('saved_epochs3')
#         folder_path = Path(epoch_path,participant_id)
#         # Check if saved_epochs folder already exists
#         if not epoch_path.exists():
#             # Create the folder
#             epoch_path.mkdir(parents=True)
#         # Check if participant folder already exists
#         if not folder_path.exists():
#             # Create the folder
#             folder_path.mkdir(parents=True)
#         # Construct the filename with condition, participant ID, and block number
#         epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
#         annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
#         # Save the epochs and annotations
#         epochs.save(folder_path / epochs_filename, overwrite=True)
#         np.save(folder_path / annotations_filename, events)    
        
        
        
    


#### Participant AO ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AO")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","UR","AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AO_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.75)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
        
        
        
    


#### Participant AP ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AP")

# conditions we are interested in as a list
conds_of_interest = ["US","UR"]
conds_of_interest2 = ["AS"]
conds_of_interest3 = ["AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AP_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID 
    block_number = file_parts[2]  # Block number (e.g., "01")

    
    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
                     
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        # raw.drop_channels(['CP2'])
        raw.drop_channels(['CP6'])
        # raw.plot()
        raw.set_montage('standard_1020')
     
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 
    if file_cond in conds_of_interest3 and block_number == "01":
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels and 
        raw.pick("eeg")
        raw.drop_channels(['CP6'])
        # raw.plot()
        raw.set_montage('standard_1020')

        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)     
    if file_cond in conds_of_interest3 and block_number == "02":
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels and 
        raw.pick("eeg")
        # raw.drop_channels(['Oz'])
        raw.drop_channels(['CP2']) ########################
        
        # raw.plot()
        raw.set_montage('standard_1020')
 
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)
        
        
        
    


#### Participant AQ ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AQ")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","UR","AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AQ_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID 
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
        
        
        
    


#### Participant AR ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AR")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","UR","AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AR_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        ####
        # orig_events, _ = mne.events_from_annotations(raw)
        # events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        # print(events)
        
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
        
        
        
    


#### Participant AS ####

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data3\\AS")

# conditions we are interested in as a list
conds_of_interest = ["US","AS","AR"]
conds_of_interest2 = ["UR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AS_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 
    if file_cond in conds_of_interest2 and block_number == '01': ### block 2 was misrecorded
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
    
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = orig_events
        events.shape
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs3')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)   
        
        
        
    


### Read evoked data of dataset 2022 ###

In [3]:
def load_and_combine_epochs_for_condition(part_directory, condition):
    epochs_list = []
    for block_file in part_directory.glob(f"{condition}_{part_directory.stem}_*-epo.fif"):
        epochs = mne.read_epochs(block_file, preload=True)
        # Check if this is participant "AT" and adjust channels
        if part_directory.stem == "AT":
            if condition == "US":
                channels_to_drop = [ch for ch in ['FC6', 'CP2'] if ch in epochs.ch_names]
                epochs.drop_channels(channels_to_drop)
        if part_directory.stem == "AJ":
            if condition == "US":
                channels_to_drop = [ch for ch in ['FC6'] if ch in epochs.ch_names]
                epochs.drop_channels(channels_to_drop)
        if part_directory.stem == "AP":
            if condition == "AS":
                channels_to_drop = [ch for ch in ['CP6'] if ch in epochs.ch_names]
                epochs.drop_channels(channels_to_drop)
            if condition == "AR":
                channels_to_drop = [ch for ch in ['CP6', 'CP2'] if ch in epochs.ch_names]
                epochs.drop_channels(channels_to_drop)
        epochs_list.append(epochs)
    combined_epochs = mne.concatenate_epochs(epochs_list)
    return combined_epochs

conds_of_interest = ["US","AS","UR","AR"]

# Main processing loop
all_parts22 = {}
for part in Path("saved_epochs3").iterdir():
    evokeds = []
    for cond in conds_of_interest:
        # Load and combine epochs for each condition across blocks
        combined_epochs = load_and_combine_epochs_for_condition(part, cond)
        
        if "A" in cond:
            lat_cond = [f"{cond}/lat_am"]
        elif "U" in cond:
            lat_cond = [f"{cond}/lat_unam_r", f"{cond}/lat_unam_l"]

        evoked = combined_epochs[lat_cond].average()
        evoked.comment = cond  # Simplify condition name directly here
        evokeds.append(evoked)

    all_parts22[part.stem] = evokeds

def simplify_condition(comment):
    for cond in conds_of_interest:
        if cond in comment:
            return cond
    return None

def simplify_evoked_conditions(participant_evokeds):
    simplified_evokeds = []
    for evoked in participant_evokeds:
        simplified_comment = simplify_condition(evoked.comment)
        if simplified_comment:
            evoked.comment = simplified_comment
            simplified_evokeds.append(evoked)
    return simplified_evokeds
for participant in list(all_parts22):
    all_parts22[participant] = simplify_evoked_conditions(all_parts22[participant])

Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AJ\US_AJ_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
215 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AJ\US_AJ_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
190 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
405 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AJ\AS_AJ_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
301 matching events found
No baseline correction applied
0

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\2452503459.py:22: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
254 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
555 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AJ\UR_AJ_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\2452503459.py:22: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


255 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AJ\UR_AJ_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
559 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AJ\AR_AJ_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata


...

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\2452503459.py:22: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AT\UR_AT_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
271 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
571 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AT\AR_AT_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\2452503459.py:22: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


Not setting metadata
281 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs3\AT\AR_AT_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
581 matching events found
Applying baseline correction (mode: mean)


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\2452503459.py:22: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


# Dataset 2021 #

## Preprocessing ##

### Trigger control ###

In [5]:
"""Convert old trigger system to new one"""

def get_new_events_old_trigger_system(orig_events, sfreq):
    events = orig_events.copy()
    samples = 1000 / sfreq
    small_check_normal = 300 / samples
    small_check_rev = 250 / samples
    amb_lat = 200 / samples
    unamb_left_lat = 100 / samples
    unamb_right_lat = 150 / samples

    # Define a dictionary to map old triggers to new ones
    trigger_map = {
        small_check_normal: 2,
        small_check_rev: 4,
        amb_lat: 6,
        unamb_left_lat: 8,
        unamb_right_lat: 10
        
    }

    # Ignore initial values of 99999 and anything between 1000-1100
    valid_start = np.argwhere((events[:, 2] == 13) & (events[:, 2] < 1000) | (events[:, 2] > 1100))
    if valid_start.size == 0:
        return events  # No valid start found, return original events

    start_index = valid_start[0][0]

    # Find all 13 triggers starting from the first valid start index
    trig_13_indices = np.where(events[start_index:, 2] == 13)[0] + start_index
    # Find all 12 triggers starting from the first valid start index
    trig_12_indices = np.where(events[start_index:, 2] == 12)[0] + start_index

    # Ensure there is a matching 12 for each 13
    for idx_13 in trig_13_indices:
        following_12 = trig_12_indices[trig_12_indices > idx_13]
        if not following_12.size:
            continue  # If there's no matching 12, skip this 13

        idx_12 = following_12[0]
        trigger_length = events[idx_12, 0] - events[idx_13, 0]
        closest_trigger = min(trigger_map, key=lambda x: abs(x - trigger_length))
        events[idx_13, 2] = trigger_map[closest_trigger]

    return events


In [ ]:
# Check triggers ###
"""-> AI is excluded"""
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AI")

# conditions we are interested in as a list
conds_of_interest = ["AS"]

# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5
counter = 0
all_event_codes = []
all_event_codes2 = []
# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AI_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest and block_number == '02':
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        def check_event_pairs(orig_events):
            paired_count = 0
            unpaired_13 = []
            unpaired_12 = []
            ignore_codes = set(range(1000, 1101))
            orig_events = np.array(orig_events)
            # Look for 13 events and check if the next event is 12
            for i, event in enumerate(orig_events):
                if event[2] == 13:
                    # Initialize a variable to keep track of the next relevant index
                    next_relevant_index = i + 1
                    
                    # Skip any event codes between 1010 and 1100
                    while next_relevant_index < len(orig_events) and orig_events[next_relevant_index][2] in ignore_codes:
                        next_relevant_index += 1
                        
                    # If the next relevant event is 12, count as a pair
                    if next_relevant_index < len(orig_events) and orig_events[next_relevant_index][2] == 12:
                        paired_count += 1
                    else:
                        # Otherwise, the 13 event is unpaired
                        unpaired_13.append(i)
                        
                elif event[2] == 12:
                    # Check if the previous event is a 13 and not unpaired
                    prev_relevant_index = i - 1
                    
                    # Skip any event codes between 1010 and 1100
                    while prev_relevant_index >= 0 and orig_events[prev_relevant_index][2] in ignore_codes:
                        prev_relevant_index -= 1
                        
                    # If the previous relevant event is not a 13, count this 12 as unpaired
                    if prev_relevant_index < 0 or orig_events[prev_relevant_index][2] != 13:
                        unpaired_12.append(i)

            return paired_count, unpaired_13, unpaired_12



        # Now let's call the function and print the results
        paired_count, unpaired_13s, unpaired_12s = check_event_pairs(orig_events)
        print(f"Number of correctly paired '13' followed by '12': {paired_count}")
        print(f"Unpaired '13' indices: {unpaired_13s}")
        print(f"Unpaired '12' indices: {unpaired_12s}")
            
        # Find indices of triggers 12 and 13
        trig_12_indices = np.where(orig_events[:, 2] == 12)[0]
        trig_13_indices = np.where(orig_events[:, 2] == 13)[0]
            
        # Iterate through the indices of triggers 13
        for trig_12_index in trig_12_indices:
            # Find the corresponding trigger 12 that comes before the current trigger 13
            previous_trig_13_index = trig_13_indices[trig_13_indices < trig_12_index]
            if previous_trig_13_index.size > 0:  # Check if a previous trigger 12 exists
                previous_trig_13_index = previous_trig_13_index[-1]
                # Calculate the difference in sample points
                difference = orig_events[trig_12_index, 0] - orig_events[previous_trig_13_index, 0]
                print(f"Difference {counter + 1}: {difference}")
                # Add the difference to the list
                # sample_differences.append(difference)
                # Increment the counter
                counter += 1
                if counter == 20:  # Stop after printing 20 differences
                    break
            if counter == 20:  # Check if we need to break out of the outer loop as well
                break
        
        
    


#### Participant AA ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AA")

# conditions we are interested in as a list
conds_of_interest = ["AS","US","UR"]
conds_of_interest2 = ["AR"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

all_event_codes = []
all_event_codes2 = []
# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AA_*.vhdr"):  ### has only one block
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.D., "01")

    if file_cond in conds_of_interest: 
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
 
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        # raw.plot()
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
        events.shape
        # Plot events
        # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
        # so cut raw into epochs
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs2')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        raw.drop_channels(['FC6']) ## noisy channel
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
 
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        # raw.plot()
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
        events.shape
        # Plot events
        # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
        # so cut raw into epochs
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        # epochs.plot()
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot()
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs2')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)


#### Participant AB ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AB")

# conditions we are interested in as a list
conds_of_interest = ["AS","AR","US","UR"]

# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

all_event_codes = []
all_event_codes2 = []
# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AB_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
        events.shape
        # Plot events
        # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
        # so cut raw into epochs
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs2')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)


#### Participant AC ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AC")

# conditions we are interested in as a list
conds_of_interest = ["AS","AR","US","UR"]

# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

all_event_codes = []
all_event_codes2 = []
# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AC_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
        events.shape
        # Plot events
        # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
   
        # so cut raw into epochs
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs2')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)


#### Participant AD ####

In [6]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AD")

# conditions we are interested in as a list
conds_of_interest = ["AS","AR","US","UR"]

# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AD_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest: ### block 01 was excluded, block 2 and 3 were analyzed
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')

        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
        events.shape
        # Plot events
        # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
        # so cut raw into epochs
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.8)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs2')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)


Extracting parameters from C:\Users\mtu10\OneDrive\Desktop\EEG_code\Data2\AD\AR_AD_01.vhdr...
Setting channel info structure...
Reading 0 ... 157099  =      0.000 ...   157.099 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.03 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.03
- Lower transition bandwidth: 0.03 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 110001 samples (110.001 s)



C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\3396187499.py:38: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(file,preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Used Annotations descriptions: ['New Segment/', 'Response/R  7', 'Response/R 15', 'Stimulus/S 12', 'Stimulus/S 13']
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 304 events and 601 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    2.3s remaining:   21.7s
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:    2.4s remaining:    7.5s
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed:    2.4s remaining:    3.7s
[Parallel(n_jobs=20)]: Done  11 out of  20 | elapsed:    2.5s remaining:    2.0s
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed:    2.6s remaining:    1.0s
[Parallel(n_jobs=20)]: Done  17 out of  20 | elapsed:    2.7s remaining:    0.4s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    2.7s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.9s remaining:    9.1s
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:    1.0s remaining:    3.1s
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed:    1.0s remaining:    1.6s
[Parallel(n_jobs=20)]: Done  11 out of  20 | e

[Done]
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 96.0 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 1 sensors
Cz
Fitting ICA to data using 31 channels (please be patient, this may take a while)
Selecting by number: 30 components
Fitting ICA took 6.8s.
Using EOG channels: Fp1, Fp2
Applying ICA to Epochs instance
    Transforming to ICA space (30 components)
    Zeroing out 0 ICA components
    Projecting back using 31 PCA components
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Applying baseline correction (mode: mean)
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch

...

Used Annotations descriptions: ['New Segment/', 'Response/R  5', 'Response/R  7', 'Response/R 13', 'Response/R 15', 'Stimulus/S 12', 'Stimulus/S 13']
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 304 events and 601 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Batch computation too fast (0.07461881637573242s.) Setting batch_size=2.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.0s remaining:    0.6s
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=20)]: Done   8 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  11 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  17 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.9s remaining:    9.3s
[Parallel(n_jobs=20)]: Done   5 out of  20 | elapsed:    1.0s remaining:    3.2s
[Parallel(n_jobs=20)]: Done   8

[Done]
Setting channel interpolation method to {'eeg': 'spline'}.

Fitting ICA to data using 31 channels (please be patient, this may take a while)


c:\Python311\Lib\site-packages\autoreject\ransac.py:244: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  epochs.interpolate_bads(reset_bads=True)


Selecting by number: 30 components
Fitting ICA took 4.0s.
Using EOG channels: Fp1, Fp2
Applying ICA to Epochs instance
    Transforming to ICA space (30 components)
    Zeroing out 0 ICA components
    Projecting back using 31 PCA components
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Applying baseline correction (mode: mean)
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
    Rejecting  epoch based on EEG : ['Fp1', 'Fp2']
12 bad epochs dropped

#### Participant AE ####

In [ ]:
"""AE was excluded"""

# # get data path to specific participant
# participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AE")

# # conditions we are interested in as a list
# conds_of_interest = ["AS","US","UR"]
# conds_of_interest2 = ["AR"]
# # tmin and tmax of epochs
# tmin = -0.1
# tmax = 0.5

# all_event_codes = []
# all_event_codes2 = []
# # Participant name
# participant_id = participant_path.stem

# for file in participant_path.glob("*_AE_*.vhdr"):
#     # Extract the condition and block from the file name 
#     file_parts = file.stem.split('_')
#     file_cond = file_parts[0]  # Condition (e.g., "AR")
#     participant_id = file_parts[1]  # Participant ID (e.g., "AB")
#     block_number = file_parts[2]  # Block number (e.g., "01")

#     if file_cond in conds_of_interest:
#         # event triggers as dict used file cond info
#         event_id_am = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_am": 6}
#         event_id_unam = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_unam_l": 8,
#                     f"{file_cond}/lat_unam_r": 10}
#         # get event_id corresponding with condition
#         if "A" in file_cond:
#             event_id = event_id_am
#         elif "U" in file_cond:
#             event_id = event_id_unam
        
                    
#        # load raw data
#         raw = mne.io.read_raw_brainvision(file,preload=True)
#         # only pick eeg channels
#         raw.pick("eeg")
        
#         # raw.plot()
#         raw.set_montage('standard_1020')
#         ####
#         # orig_events, _ = mne.events_from_annotations(raw)
#         # events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
#         # print(events)
        
        
#         # filter data
#         raw.filter(0.03,25)
#         # raw.plot_psd(fmax=40)
#         raw.pick(["eeg"]).load_data()
#         # raw.info
#         # raw.plot_sensors(show_names=True)
#         # fig = raw.plot_sensors("3d")
#         # fig
        
#         # get triggers
#         orig_events, _ = mne.events_from_annotations(raw)
#         events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
#         events.shape
#         # Plot events
#         # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
#         # find band channels and interpolate
#         # code only works if data is in epochs
#         # so cut raw into epochs
#         epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

#         picks = mne.pick_types(epochs.info, meg=False, eeg=True,
#         stim=False, eog=False,
#         include=[], exclude=[])
#         ransac = Ransac(picks=picks, n_jobs=-1)
#         epochs = ransac.fit_transform(epochs)
#         print('\n'.join(ransac.bad_chs_))
        
#         # epochs.plot()
        
#         # compute an ICA to remove eye blink components from signal
#         ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
#         ica.fit(epochs)
#         ica.exclude = []
#         eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
#         ica.exclude = eog_idx
#         # ica.plot_scores(scores)
#         # ica.plot_components()
#         # ica.plot_sources(epochs)
#         epochs_new = epochs.copy()
#         ica.apply(epochs_new)
#         epochs = epochs_new
        
#         # set offline reference at mastoid electrodes
#         epochs.set_eeg_reference(['TP9','TP10'], projection=False)
#         # apply baseline in ms
#         epochs.apply_baseline((-0.1,0))
#         # reject epochs that are above a certain threshold
#         # value needs to be in volts
#         epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
#         # epochs.plot_drop_log()
#         # save epoch objects and events array
#         epoch_path = Path('saved_epochs2')
#         folder_path = Path(epoch_path,participant_id)
#         # Check if saved_epochs folder already exists
#         if not epoch_path.exists():
#             # Create the folder
#             epoch_path.mkdir(parents=True)
#         # Check if participant folder already exists
#         if not folder_path.exists():
#             # Create the folder
#             folder_path.mkdir(parents=True)
#         # Construct the filename with condition, participant ID, and block number
#         epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
#         annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
#         # Save the epochs and annotations
#         epochs.save(folder_path / epochs_filename, overwrite=True)
#         np.save(folder_path / annotations_filename, events)
#     if file_cond in conds_of_interest2 and block_number == "01":
#         # event triggers as dict used file cond info
#         event_id_am = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_am": 6}
#         event_id_unam = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_unam_l": 8,
#                     f"{file_cond}/lat_unam_r": 10}
#         # get event_id corresponding with condition
#         if "A" in file_cond:
#             event_id = event_id_am
#         elif "U" in file_cond:
#             event_id = event_id_unam
        
                    
#        # load raw data
#         raw = mne.io.read_raw_brainvision(file,preload=True)
#         # only pick eeg channels
#         raw.pick("eeg")
        
#         # raw.plot()
#         raw.set_montage('standard_1020')
#         ####
#         # orig_events, _ = mne.events_from_annotations(raw)
#         # events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
#         # print(events)
        
        
#         # filter data
#         raw.filter(0.03,25)
#         # raw.plot_psd(fmax=40)
#         raw.pick(["eeg"]).load_data()
#         # raw.info
#         # raw.plot_sensors(show_names=True)
#         # fig = raw.plot_sensors("3d")
#         # fig
        
#         # get triggers
#         orig_events, _ = mne.events_from_annotations(raw)
#         events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
#         events.shape
#         # Plot events
#         # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
#         # find band channels and interpolate
#         # code only works if data is in epochs
#         # so cut raw into epochs
#         epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

#         picks = mne.pick_types(epochs.info, meg=False, eeg=True,
#         stim=False, eog=False,
#         include=[], exclude=[])
#         ransac = Ransac(picks=picks, n_jobs=-1)
#         epochs = ransac.fit_transform(epochs)
#         print('\n'.join(ransac.bad_chs_))
        
#         # epochs.plot()
        
#         # compute an ICA to remove eye blink components from signal
#         ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
#         ica.fit(epochs)
#         ica.exclude = []
#         eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
#         ica.exclude = eog_idx
#         # ica.plot_scores(scores)
#         # ica.plot_components()
#         # ica.plot_sources(epochs)
#         epochs_new = epochs.copy()
#         ica.apply(epochs_new)
#         epochs = epochs_new
        
#         # set offline reference at mastoid electrodes
#         epochs.set_eeg_reference(['TP9','TP10'], projection=False)
#         # apply baseline in ms
#         epochs.apply_baseline((-0.1,0))
#         # reject epochs that are above a certain threshold
#         # value needs to be in volts
#         epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
#         # epochs.plot_drop_log()
#         # save epoch objects and events array
#         epoch_path = Path('saved_epochs2')
#         folder_path = Path(epoch_path,participant_id)
#         # Check if saved_epochs folder already exists
#         if not epoch_path.exists():
#             # Create the folder
#             epoch_path.mkdir(parents=True)
#         # Check if participant folder already exists
#         if not folder_path.exists():
#             # Create the folder
#             folder_path.mkdir(parents=True)
#         # Construct the filename with condition, participant ID, and block number
#         epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
#         annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
#         # Save the epochs and annotations
#         epochs.save(folder_path / epochs_filename, overwrite=True)
#         np.save(folder_path / annotations_filename, events)


#### Participant AF ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AF")

# conditions we are interested in as a list
conds_of_interest = ["AS","AR","US","UR"]

# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5


# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AF_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
      
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
        events.shape
        # Plot events
        # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
        # so cut raw into epochs
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs2')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)


#### Participant AH ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AH")

# conditions we are interested in as a list
conds_of_interest = ["AS","AR","US","UR"]

# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AH_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
  
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
        events.shape
        # Plot events
        # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
      
        # cut raw into epochs
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.75)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs2')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)


#### Participant AG ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AG")

# conditions we are interested in as a list
conds_of_interest = ["AS","AR","US","UR"]

# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5


# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_AG_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
        events.shape
        # Plot events
        # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
    
        # cut raw into epochs
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.65)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs2')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)


#### Participant AI ####

In [ ]:
"""--> AI was excluded"""
# # get data path to specific participant
# participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data2\\AI")

# # conditions we are interested in as a list
# conds_of_interest = ["AR","US","UR"]
# conds_of_interest2 = ["AS"] 
# # tmin and tmax of epochs
# tmin = -0.1
# tmax = 0.5

# # Participant name
# participant_id = participant_path.stem

# for file in participant_path.glob("*_AI_*.vhdr"):
#     # Extract the condition and block from the file name 
#     file_parts = file.stem.split('_')
#     file_cond = file_parts[0]  # Condition (e.g., "AR")
#     participant_id = file_parts[1]  # Participant ID (e.g., "AB")
#     block_number = file_parts[2]  # Block number (e.g., "01")

#     if file_cond in conds_of_interest:
#         # event triggers as dict used file cond info
#         event_id_am = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_am": 6}
#         event_id_unam = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_unam_l": 8,
#                     f"{file_cond}/lat_unam_r": 10}
#         # get event_id corresponding with condition
#         if "A" in file_cond:
#             event_id = event_id_am
#         elif "U" in file_cond:
#             event_id = event_id_unam
        
                    
#        # load raw data
#         raw = mne.io.read_raw_brainvision(file,preload=True)
#         # only pick eeg channels
#         raw.pick("eeg")
        
#         # raw.plot()
#         raw.set_montage('standard_1020')
#         ####
#         # orig_events, _ = mne.events_from_annotations(raw)
#         # events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
#         # print(events)
        
        
#         # filter data
#         raw.filter(0.03,25)
#         # raw.plot_psd(fmax=40)
#         raw.pick(["eeg"]).load_data()
#         # raw.info
#         # raw.plot_sensors(show_names=True)
#         # fig = raw.plot_sensors("3d")
#         # fig
        
#         # get triggers
#         orig_events, _ = mne.events_from_annotations(raw)
#         events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
#         events.shape
#         # Plot events
#         # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
#         # find band channels and interpolate
#         # code only works if data is in epochs
#         # so cut raw into epochs
#         epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

#         picks = mne.pick_types(epochs.info, meg=False, eeg=True,
#         stim=False, eog=False,
#         include=[], exclude=[])
#         ransac = Ransac(picks=picks, n_jobs=-1)
#         epochs = ransac.fit_transform(epochs)
#         print('\n'.join(ransac.bad_chs_))
        
#         # epochs.plot()
        
#         # compute an ICA to remove eye blink components from signal
#         ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
#         ica.fit(epochs)
#         ica.exclude = []
#         eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.65)
#         ica.exclude = eog_idx
#         # ica.plot_scores(scores)
#         # ica.plot_components()
#         # ica.plot_sources(epochs)
#         epochs_new = epochs.copy()
#         ica.apply(epochs_new)
#         epochs = epochs_new
        
#         # set offline reference at mastoid electrodes
#         epochs.set_eeg_reference(['TP9','TP10'], projection=False)
#         # apply baseline in ms
#         epochs.apply_baseline((-0.1,0))
#         # reject epochs that are above a certain threshold
#         # value needs to be in volts
#         epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
#         # epochs.plot_drop_log()
#         # save epoch objects and events array
#         epoch_path = Path('saved_epochs2')
#         folder_path = Path(epoch_path,participant_id)
#         # Check if saved_epochs folder already exists
#         if not epoch_path.exists():
#             # Create the folder
#             epoch_path.mkdir(parents=True)
#         # Check if participant folder already exists
#         if not folder_path.exists():
#             # Create the folder
#             folder_path.mkdir(parents=True)
#         # Construct the filename with condition, participant ID, and block number
#         epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
#         annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
#         # Save the epochs and annotations
#         epochs.save(folder_path / epochs_filename, overwrite=True)
#         np.save(folder_path / annotations_filename, events)
        
#     if file_cond in conds_of_interest2 and block_number == "01":
#         # event triggers as dict used file cond info
#         event_id_am = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_am": 6}
#         event_id_unam = {f"{file_cond}/checks_normal": 2,
#                     f"{file_cond}/checks_rev": 4,
#                     f"{file_cond}/lat_unam_l": 8,
#                     f"{file_cond}/lat_unam_r": 10}
#         # get event_id corresponding with condition
#         if "A" in file_cond:
#             event_id = event_id_am
#         elif "U" in file_cond:
#             event_id = event_id_unam
        
                    
#        # load raw data
#         raw = mne.io.read_raw_brainvision(file,preload=True)
#         # only pick eeg channels
#         raw.pick("eeg")
        
#         # raw.plot()
#         raw.set_montage('standard_1020')
#         ####
#         # orig_events, _ = mne.events_from_annotations(raw)
#         # events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
#         # print(events)
        
        
#         # filter data
#         raw.filter(0.03,25)
#         # raw.plot_psd(fmax=40)
#         raw.pick(["eeg"]).load_data()
#         # raw.info
#         # raw.plot_sensors(show_names=True)
#         # fig = raw.plot_sensors("3d")
#         # fig
        
#         # get triggers
#         orig_events, _ = mne.events_from_annotations(raw)
#         events = get_new_events_old_trigger_system(orig_events, raw.info["sfreq"]) #####
        
#         events.shape
#         # Plot events
#         # mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])
        
#         # find band channels and interpolate
#         # code only works if data is in epochs
#         # so cut raw into epochs
#         epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None,tmin=tmin,tmax=tmax,proj=True,preload=True)

#         picks = mne.pick_types(epochs.info, meg=False, eeg=True,
#         stim=False, eog=False,
#         include=[], exclude=[])
#         ransac = Ransac(picks=picks, n_jobs=-1)
#         epochs = ransac.fit_transform(epochs)
#         print('\n'.join(ransac.bad_chs_))
        
#         # epochs.plot()
        
#         # compute an ICA to remove eye blink components from signal
#         ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
#         ica.fit(epochs)
#         ica.exclude = []
#         eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.65)
#         ica.exclude = eog_idx
#         # ica.plot_scores(scores)
#         # ica.plot_components()
#         # ica.plot_sources(epochs)
#         epochs_new = epochs.copy()
#         ica.apply(epochs_new)
#         epochs = epochs_new
        
#         # set offline reference at mastoid electrodes
#         epochs.set_eeg_reference(['TP9','TP10'], projection=False)
#         # apply baseline in ms
#         epochs.apply_baseline((-0.1,0))
#         # reject epochs that are above a certain threshold
#         # value needs to be in volts
#         epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
#         # epochs.plot_drop_log()
#         # save epoch objects and events array
#         epoch_path = Path('saved_epochs2')
#         folder_path = Path(epoch_path,participant_id)
#         # Check if saved_epochs folder already exists
#         if not epoch_path.exists():
#             # Create the folder
#             epoch_path.mkdir(parents=True)
#         # Check if participant folder already exists
#         if not folder_path.exists():
#             # Create the folder
#             folder_path.mkdir(parents=True)
#         # Construct the filename with condition, participant ID, and block number
#         epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
#         annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
#         # Save the epochs and annotations
#         epochs.save(folder_path / epochs_filename, overwrite=True)
#         np.save(folder_path / annotations_filename, events)    
        
        
       
        


    


## Reading Evoked data 2021 ##

In [4]:

def load_and_combine_epochs_for_condition(part_directory, condition):
    epochs_list = []
    for block_file in part_directory.glob(f"{condition}_{part_directory.stem}_*-epo.fif"):
        epochs = mne.read_epochs(block_file, preload=True)
        # Cadjust channels
        if part_directory.stem == "AA":
            # Drop specific channels if they are present
            channels_to_drop = [ch for ch in ['FC6'] if ch in epochs.ch_names]
            epochs.drop_channels(channels_to_drop)
        epochs_list.append(epochs)
    combined_epochs = mne.concatenate_epochs(epochs_list)
    return combined_epochs

conds_of_interest = ["US","AS","UR","AR"]
# Main processing loop
all_parts21 = {}
for part in Path("saved_epochs2").iterdir():
    evokeds = []
    for cond in conds_of_interest:
        # Load and combine epochs for each condition across blocks
        combined_epochs = load_and_combine_epochs_for_condition(part, cond)
        
        if "A" in cond:
            lat_cond = [f"{cond}/lat_am"]
        elif "U" in cond:
            lat_cond = [f"{cond}/lat_unam_r", f"{cond}/lat_unam_l"]

        evoked = combined_epochs[lat_cond].average()
        evoked.comment = cond  # Simplify condition name directly here
        evokeds.append(evoked)

    all_parts21[part.stem] = evokeds


colors = {
    'US': 'red',
    'AS': 'orange',
    'AR': 'blue',
    'UR': 'green'
}

def simplify_condition(comment):
    for cond in conds_of_interest:
        if cond in comment:
            return cond
    return None

def simplify_evoked_conditions(participant_evokeds):
    simplified_evokeds = []
    for evoked in participant_evokeds:
        simplified_comment = simplify_condition(evoked.comment)
        if simplified_comment:
            evoked.comment = simplified_comment
            simplified_evokeds.append(evoked)
    return simplified_evokeds
for participant in list(all_parts21):
    all_parts21[participant] = simplify_evoked_conditions(all_parts21[participant])

Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AA\US_AA_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
399 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
399 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AA\AS_AA_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
400 matching events found
No baseline correction applied
0 projection items activated


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1301839981.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1301839981.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


Not setting metadata
400 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AA\UR_AA_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
400 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
400 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AA\AR_AA_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
400 matching events found
No baseline correction applied
0 projection items activated


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1301839981.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1301839981.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


Not setting metadata
400 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AB\US_AB_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AB\US_AB_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
608 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AB\AS_AB_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices 

...

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1301839981.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


Not setting metadata
292 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AH\UR_AH_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
302 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
594 matching events found
Applying baseline correction (mode: mean)
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AH\AR_AH_01-epo.fif ...
    Found the data of interest:


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1301839981.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
303 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs2\AH\AR_AH_02-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
298 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
601 matching events found
Applying baseline correction (mode: mean)


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1301839981.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  combined_epochs = mne.concatenate_epochs(epochs_list)


In [6]:
all_parts21

{'AJ': [<Evoked | 'US' (average, N=136), -0.1 – 0.5 s, baseline -0.1 – 0 s, 30 ch, ~183 kB>,
  <Evoked | 'AS' (average, N=151), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'UR' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AR' (average, N=142), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>],
 'AK': [<Evoked | 'US' (average, N=146), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AS' (average, N=150), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'UR' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AR' (average, N=151), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>],
 'AL': [<Evoked | 'US' (average, N=150), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AS' (average, N=124), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'UR' (average, N=142), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AR' (avera

# DATA 2020 #

## Trigger conversion ##

In [ ]:
def get_new_events_old_trigger_system2(orig_events, sfreq):
    '''Change event triggers from duration to different trigger numbers depending on stimulus type.
    Determined by sample points.
    Parameters:
    -----------------
    events: numpy.ndarray, array from mne funtion events_from_annotations 
    sfreq: float, sample frequency
    Output:
    ----------------
    events: numpy.ndarray, array with new trigger values
    '''
    events = orig_events.copy()
    #find where stimulus trigger is 13 because those are stim onset
    trig_13 = np.where(orig_events[:,-1]==13)[0]
    # find where trigger is 12 because that is stim offset
    trig_12 = np.where(orig_events[:,-1]==12)[0]
    
    #calculate number of samples in the case that the sample frequency is not 1000
    #divide original sample frequency by new sample frequency
    samples = 1000/sfreq
    #divide original number of samples for stim by samples calculated above
    small_check_normal=300/samples
    small_check_rev=250/samples
    amb_lat=200/samples
    unamb_left_lat=100/samples
    unamb_right_lat=150/samples
    # all trigger lengths as list
    all_trig_lens = [small_check_normal, small_check_rev, amb_lat, unamb_left_lat, unamb_right_lat]
    
    ## change trigger numbers
    #loop through trigger indices
    for i in range(len(trig_12)):
        # find trigger duration
        trig_len = orig_events[trig_12[i],0]-orig_events[trig_13[i],0]
        
        # compare all computed trigger lengths with actual trigger lengths
        # put differences of actual trigger and computer trigger lengths into list
        abs_trig_len = []
        for t in all_trig_lens:
            abs_trig_len.append(abs(t-trig_len))
        
        # find the trigger length with smallest difference between computed and actual trigger length
        diff_min = np.argmin(abs_trig_len)
        # get real trigger value
        real_trigger_len = all_trig_lens[diff_min]
        # get different trig values for diff stimuli
        if real_trigger_len == small_check_normal:
            new_trig = 2
        elif real_trigger_len == small_check_rev:
            new_trig = 4
        elif real_trigger_len == amb_lat:
            new_trig = 6
        elif real_trigger_len == unamb_left_lat:
            new_trig = 8
        elif real_trigger_len == unamb_right_lat:
            new_trig = 10
            
        # replace trigger 13 with values
        events[trig_13[i],2] = new_trig    
            
    return events

### Trigger checking ###

In [ ]:

# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data\\KM")

# conditions we are interested in as a list
conds_of_interest = ["AR"]

# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5
counter = 0
all_event_codes = []
all_event_codes2 = []
# Participant name
participant_id = participant_path.stem

for file in participant_path.glob("*_KM_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.g., "01")

    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        
                    
       # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        def check_event_pairs(orig_events):
            paired_count = 0
            unpaired_13 = []
            unpaired_12 = []
            ignore_codes = set(range(1000, 1101))
            orig_events = np.array(orig_events)
            # Look for 13 events and check if the next event is 12
            for i, event in enumerate(orig_events):
                if event[2] == 13:
                    # Initialize a variable to keep track of the next relevant index
                    next_relevant_index = i + 1
                    
                    # Skip any event codes between 1010 and 1100
                    while next_relevant_index < len(orig_events) and orig_events[next_relevant_index][2] in ignore_codes:
                        next_relevant_index += 1
                        
                    # If the next relevant event is 12, count as a pair
                    if next_relevant_index < len(orig_events) and orig_events[next_relevant_index][2] == 12:
                        paired_count += 1
                    else:
                        # Otherwise, the 13 event is unpaired
                        unpaired_13.append(i)
                        
                elif event[2] == 12:
                    # Check if the previous event is a 13 and not unpaired
                    prev_relevant_index = i - 1
                    
                    # Skip any event codes between 1010 and 1100
                    while prev_relevant_index >= 0 and orig_events[prev_relevant_index][2] in ignore_codes:
                        prev_relevant_index -= 1
                        
                    # If the previous relevant event is not a 13, count this 12 as unpaired
                    if prev_relevant_index < 0 or orig_events[prev_relevant_index][2] != 13:
                        unpaired_12.append(i)

            return paired_count, unpaired_13, unpaired_12



        # Now let's call the function and print the results
        paired_count, unpaired_13s, unpaired_12s = check_event_pairs(orig_events)
        print(f"Number of correctly paired '13' followed by '12': {paired_count}")
        print(f"Unpaired '13' indices: {unpaired_13s}")
        print(f"Unpaired '12' indices: {unpaired_12s}")
            
        # Find indices of triggers 12 and 13
        trig_12_indices = np.where(orig_events[:, 2] == 12)[0]
        trig_13_indices = np.where(orig_events[:, 2] == 13)[0]
            
        # Iterate through the indices of triggers 13
        for trig_12_index in trig_12_indices:
            # Find the corresponding trigger 12 that comes before the current trigger 13
            previous_trig_13_index = trig_13_indices[trig_13_indices < trig_12_index]
            if previous_trig_13_index.size > 0:  # Check if a previous trigger 12 exists
                previous_trig_13_index = previous_trig_13_index[-1]
                # Calculate the difference in sample points
                difference = orig_events[trig_12_index, 0] - orig_events[previous_trig_13_index, 0]
                print(f"Difference {counter + 1}: {difference}")
                # Add the difference to the list
                # sample_differences.append(difference)
                # Increment the counter
                counter += 1
                if counter == 20:  # Stop after printing 20 differences
                    break
            if counter == 20:  # Check if we need to break out of the outer loop as well
                break
        
        
    


### Filter repetitive Necker lattices ###

In [ ]:
def filter_event_sequences(events): ### Detect repetitive necker lattices (no checkerboards in between)
    filtered_event_codes = []
    filtered_event_indices = []
    ignore_codes = {12, 1007, 1011, 1013, 1015}
    sequence_start_codes = {6, 8, 10}
    valid_terminators = {2, 4}
    
    skip_until_valid = False
    last_sequence_code = None

    for i, event in enumerate(events):
        code = event[2]
        if code in ignore_codes:
            continue
        elif skip_until_valid and code in sequence_start_codes and last_sequence_code == code:
            continue
        elif skip_until_valid and code in valid_terminators:
            skip_until_valid = False
            filtered_event_codes.append(code)
            filtered_event_indices.append(i)
        elif code in sequence_start_codes:
            if not skip_until_valid:
                filtered_event_codes.append(code)
                filtered_event_indices.append(i)
            skip_until_valid = True
            last_sequence_code = code
        else:
            filtered_event_codes.append(code)
            filtered_event_indices.append(i)

    # Return both the filtered event codes and their indices
    return filtered_event_codes, filtered_event_indices

#### Participant KM ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data\\KM")
# conditions we are interested in as a list
conds_of_interest = ["AR","UR","US"]
conds_of_interest2 = ["AS"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5
participant_id = participant_path.stem
all_event_codes_KM = []
for file in participant_path.glob("*_KM_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID (e.g., "AB")
    block_number = file_parts[2]  # Block number (e.g., "01")
    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
            
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        #raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        #raw.plot_psd(fmax=40)
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])
        filtered_event_codes, filtered_event_indices = filter_event_sequences(events)
        filtered_events = events[filtered_event_indices]

        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=filtered_events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)
                    
        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)

        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
 
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
        
        
        
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        # raw.plot()
        raw.drop_channels(['T8']) ## drop noisy channel
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        #raw.plot_psd(fmax=40)
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])


        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)
                    
        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)

        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
            
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)
        
        
        
    


#### Participant KN ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data\\KN")
participant_id = participant_path.stem
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5

conds_of_interest = ["AR"]
conds_of_interest2 = ["UR"]
conds_of_interest3 = ["AS", "US"]
all_event_codes_KN = []
for file in participant_path.glob("*_KN_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID 
    block_number = file_parts[2]  # Block number (e.g., "01")
    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        #raw.plot_psd(fmax=40)
        raw.drop_channels(['Fp2']) ## drop noisy channel
        ## get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])
        filtered_event_codes, filtered_event_indices = filter_event_sequences(events)
        filtered_events = events[filtered_event_indices]

        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=filtered_events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1'],measure='correlation',threshold=0.55) ## fp2 noisy
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)

        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
            
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
    if file_cond in conds_of_interest3:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        #raw.plot_psd(fmax=40)
    
        ## get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])
        filtered_event_codes, filtered_event_indices = filter_event_sequences(events)
        filtered_events = events[filtered_event_indices]

        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=filtered_events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)

        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
            
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    
        
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        #raw.plot()
        raw.set_montage('standard_1020')
        raw.drop_channels(['T8']) ## drop noisy channel
        # filter data
        raw.filter(0.03,25)
        #raw.plot_psd(fmax=40)
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])
        filtered_event_codes, filtered_event_indices = filter_event_sequences(events)
        filtered_events = events[filtered_event_indices]

        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=filtered_events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)
                    
        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)

        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
            
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)    

#### Participant KO ####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data\\KO")
participant_id = participant_path.stem

# conditions we are interested in as a list
conds_of_interest = ["AR","UR"]
conds_of_interest2 = ["AS","US"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5
all_event_codes_KO = []
for file in participant_path.glob("*_KO_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID 
    block_number = file_parts[2]  # Block number (e.g., "01")
    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
    
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        #raw.plot_psd(fmax=40)
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])
        filtered_event_codes, filtered_event_indices = filter_event_sequences(events)
        filtered_events = events[filtered_event_indices]

        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=filtered_events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        #ica.plot_components()
        #ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)

        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
            
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        # raw.plot()
        raw.set_montage('standard_1020')
        # filter data
        raw.filter(0.03,25)
        #raw.plot_psd(fmax=40)
        raw.drop_channels(['T8']) ## drop noisy channel
        ## get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])
        filtered_event_codes, filtered_event_indices = filter_event_sequences(events)
        filtered_events = events[filtered_event_indices]

        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=filtered_events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        ica.plot_components()
        ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)

        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
            
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)   

#### Participant KP #####

In [ ]:
# get data path to specific participant
participant_path = Path("C:\\Users\\mtu10\\OneDrive\\Desktop\\EEG_code\\Data\\KP")
participant_id = participant_path.stem

# conditions we are interested in as a list
conds_of_interest = ["AR","UR"]
conds_of_interest2 = ["US","AS"]
# tmin and tmax of epochs
tmin = -0.1
tmax = 0.5
all_event_codes_KP = []
for file in participant_path.glob("*_KP_*.vhdr"):
    # Extract the condition and block from the file name 
    file_parts = file.stem.split('_')
    file_cond = file_parts[0]  # Condition (e.g., "AR")
    participant_id = file_parts[1]  # Participant ID 
    block_number = file_parts[2]  # Block number (e.g., "01")
    if file_cond in conds_of_interest:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
 
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])
        filtered_event_codes, filtered_event_indices = filter_event_sequences(events)
        filtered_events = events[filtered_event_indices]

        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=filtered_events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
            
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events)  
    if file_cond in conds_of_interest2:
        # event triggers as dict used file cond info
        event_id_am = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_am": 6}
        event_id_unam = {f"{file_cond}/checks_normal": 2,
                    f"{file_cond}/checks_rev": 4,
                    f"{file_cond}/lat_unam_l": 8,
                    f"{file_cond}/lat_unam_r": 10}
        # get event_id corresponding with condition
        if "A" in file_cond:
            event_id = event_id_am
        elif "U" in file_cond:
            event_id = event_id_unam
        # load raw data
        raw = mne.io.read_raw_brainvision(file,preload=True)
        # only pick eeg channels
        raw.pick("eeg")
        
        # raw.plot()
        raw.set_montage('standard_1020')
    
        
        # filter data
        raw.filter(0.03,25)
        # raw.plot_psd(fmax=40)
        raw.pick(["eeg"]).load_data()
        # raw.info
        # raw.plot_sensors(show_names=True)
        # fig = raw.plot_sensors("3d")
        # fig
        
        
        # get triggers
        orig_events, _ = mne.events_from_annotations(raw)
        events = get_new_events_old_trigger_system2(orig_events, raw.info["sfreq"])
        # filtered_event_codes, filtered_event_indices = filter_event_sequences(events)
        # filtered_events = events[filtered_event_indices] ## no need to filter

        # You can now create epochs with the filtered events
        epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None, reject=None, tmin=tmin, tmax=tmax, proj=True, preload=True)

        picks = mne.pick_types(epochs.info, meg=False, eeg=True,
        stim=False, eog=False,
        include=[], exclude=[])
        ransac = Ransac(picks=picks, n_jobs=-1)
        epochs = ransac.fit_transform(epochs)
        print('\n'.join(ransac.bad_chs_))
        
        # epochs.plot()
        
        # compute an ICA to remove eye blink components from signal
        ica = mne.preprocessing.ICA(n_components=len(epochs.info['ch_names'])-1,random_state=55)
        ica.fit(epochs)
        ica.exclude = []
        eog_idx, scores = ica.find_bads_eog(epochs,ch_name=['Fp1','Fp2'],measure='correlation',threshold=0.55)
        ica.exclude = eog_idx
        # ica.plot_scores(scores)
        # ica.plot_components()
        # ica.plot_sources(epochs)
        epochs_new = epochs.copy()
        ica.apply(epochs_new)
        epochs = epochs_new
        
        # set offline reference at mastoid electrodes
        epochs.set_eeg_reference(['TP9','TP10'], projection=False)
        # apply baseline in ms
        epochs.apply_baseline((-0.1,0))
        # reject epochs that are above a certain threshold
        # value needs to be in volts
        epochs.drop_bad(reject={"eeg": 150*1e-6},flat=None)
        # epochs.plot_drop_log()
        # save epoch objects and events array
        epoch_path = Path('saved_epochs')
        folder_path = Path(epoch_path,participant_id)
        # Check if saved_epochs folder already exists
        if not epoch_path.exists():
            # Create the folder
            epoch_path.mkdir(parents=True)
        # Check if participant folder already exists
        if not folder_path.exists():
            # Create the folder
            folder_path.mkdir(parents=True)
            
        # Construct the filename with condition, participant ID, and block number
        epochs_filename = f"{file_cond}_{participant_id}_{block_number}-epo.fif"
        annotations_filename = f"{file_cond}_{participant_id}_{block_number}-anno.npy"
            
        # Save the epochs and annotations
        epochs.save(folder_path / epochs_filename, overwrite=True)
        np.save(folder_path / annotations_filename, events) 

#### READ EPOCHS DATA 2020 

In [5]:
 
# Function definitions
def read_and_process_epochs(participant_path, condition, participant_id, block):
    # Construct the file path based on participant ID, condition, and block
    epochs_file = participant_path / f"{condition}_{participant_id}_{block}-epo.fif"
    
    if not epochs_file.is_file():
        print(f"File not found: {epochs_file}")
        return None
    
    epochs = mne.read_epochs(epochs_file, preload=True)

    # Check for channel absence for specific blocks of specific participants and conditions
    if participant_id == "KM" and condition == "AS" and block == "01":
        channels_to_drop = ['T8'] if 'T8' in epochs.ch_names else []
    elif participant_id == "KN" and block == "01":
        if condition == "UR" and 'T8' in epochs.ch_names:
            channels_to_drop = ['T8']
        elif condition == "AR" and 'Fp2' in epochs.ch_names:
            channels_to_drop = ['Fp2']
        else:
            channels_to_drop = []
    else:
        channels_to_drop = []

    if channels_to_drop:
        epochs.drop_channels(channels_to_drop)

    # Select the appropriate event IDs based on condition
    lat_cond = [f"{condition}/lat_am"] if "A" in condition else [f"{condition}/lat_unam_r", f"{condition}/lat_unam_l"]
    evoked = epochs[lat_cond].average()
    evoked.comment = f"{condition}_{block}"
    
    return evoked

# Main processing
conds_of_interest = ["US", "AS", "UR", "AR"]
participants = ["KM", "KN", "KO", "KP"]  
blocks = ["01"]  

# Create a dictionary to store evoked data for each participant
all_parts20 = {participant: [] for participant in participants}

# Directory where epochs are stored
data_path = Path("saved_epochs")

# Iterate over each participant, condition, and block
for participant_id in participants:
    participant_path = data_path / participant_id
    for condition in conds_of_interest:
        for block in blocks:
            evoked = read_and_process_epochs(participant_path, condition, participant_id, block)
            if evoked:
                all_parts20[participant_id].append(evoked)


    
def simplify_condition(comment):
    for cond in conds_of_interest:
        if cond in comment:
            return cond
    return None

def simplify_evoked_conditions(participant_evokeds):
    simplified_evokeds = []
    for evoked in participant_evokeds:
        simplified_comment = simplify_condition(evoked.comment)
        if simplified_comment:
            evoked.comment = simplified_comment
            simplified_evokeds.append(evoked)
    return simplified_evokeds

# Simplify the evoked conditions for all participants
for participant in participants:
    all_parts20[participant] = simplify_evoked_conditions(all_parts20[participant])



Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs\KM\US_KM_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
400 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs\KM\AS_KM_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
400 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs\KM\UR_KM_01-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
319 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\mtu10\OneDrive\Desktop\EEG_code\saved_epochs\KM

In [6]:
# Combine all datasets
all_parts_combined = {}

# Merge data
for participant_id, data in all_parts23.items():
    if participant_id not in all_parts_combined:
        all_parts_combined[participant_id] = data
    else:
        all_parts_combined[participant_id].extend(data)

# Merge data 
for participant_id, data in all_parts20.items():
    if participant_id not in all_parts_combined:
        all_parts_combined[participant_id] = data
    else:
        all_parts_combined[participant_id].extend(data)
for participant_id, data in all_parts21.items():
    if participant_id not in all_parts_combined:
        all_parts_combined[participant_id] = data
    else:
        all_parts_combined[participant_id].extend(data)
for participant_id, data in all_parts22.items():
    if participant_id not in all_parts_combined:
        all_parts_combined[participant_id] = data
    else:
        all_parts_combined[participant_id].extend(data)
        
        


In [7]:
# Merge all except dataset 2020
all_parts_combined_M = {}


# Merge data from all_parts3
for participant_id, data in all_parts.items():
    if participant_id not in all_parts_combined_M:
        all_parts_combined_M[participant_id] = data
    else:
        all_parts_combined_M[participant_id].extend(data)
for participant_id, data in all_parts21.items():
    if participant_id not in all_parts_combined_M:
        all_parts_combined_M[participant_id] = data
    else:
        all_parts_combined_M[participant_id].extend(data)
for participant_id, data in all_parts22.items():
    if participant_id not in all_parts_combined_M:
        all_parts_combined_M[participant_id] = data
    else:
        all_parts_combined_M[participant_id].extend(data)

In [7]:
all_parts_combined

{'BA': [<Evoked | 'US' (average, N=151), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AS' (average, N=145), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'UR' (average, N=130), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AR' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>],
 'BC': [<Evoked | 'US' (average, N=89), -0.1 – 0.5 s, baseline -0.1 – 0 s, 29 ch, ~178 kB>,
  <Evoked | 'AS' (average, N=96), -0.1 – 0.5 s, baseline -0.1 – 0 s, 29 ch, ~178 kB>,
  <Evoked | 'UR' (average, N=150), -0.1 – 0.5 s, baseline -0.1 – 0 s, 29 ch, ~178 kB>,
  <Evoked | 'AR' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 29 ch, ~178 kB>],
 'BD': [<Evoked | 'US' (average, N=151), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AS' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'UR' (average, N=152), -0.1 – 0.5 s, baseline -0.1 – 0 s, 31 ch, ~189 kB>,
  <Evoked | 'AR' (average

In [8]:
"""ERP plots for all participants individually"""
def plot_evokeds_for_participant(participant, evokeds, colors):
    # Define styles for the lines
    styles = {
        'US': {'color': 'red', 'linestyle': '-'},   # Bold red for Unambiguous-Same
        'UR': {'color': 'red', 'linestyle': '--'},  # Dashed red for Unambiguous-Random
        'AS': {'color': 'blue', 'linestyle': '-'},  # Bold blue for Ambiguous-Same
        'AR': {'color': 'blue', 'linestyle': '--'}  # Dashed blue for Ambiguous-Random
    }
    
    # Determine the common channels among all evokeds for the participant
    common_channels = set(evokeds[0].ch_names)
    for evoked in evokeds[1:]:
        common_channels.intersection_update(evoked.ch_names)
    
    # Pick only the common channels for plotting
    evokeds_common_channels = []
    for ev in evokeds:
        ev.pick_channels(list(common_channels))
        evokeds_common_channels.append(ev)

    # Adjust colors and styles based on the evoked.comment
    adjusted_styles = {ev.comment: styles[ev.comment] for ev in evokeds_common_channels if ev.comment in styles}
    adjusted_colors = {ev.comment: colors[ev.comment] for ev in evokeds_common_channels if ev.comment in colors}

    # Plotting using adjusted styles and colors
    fig, ax = plt.subplots()
    mne.viz.plot_compare_evokeds(
        {ev.comment: ev for ev in evokeds_common_channels},
        title=f"Participant '{participant}' ERPs",
        picks="Cz",
        colors=adjusted_colors,
        styles=adjusted_styles,
        ci=True,
        show=True,
        truncate_yaxis=False,
        truncate_xaxis=False,
        axes=ax
    )
   

    # Extending the x-axis to show time until 0.5 seconds
    ax.set_xlim(left=ax.get_xlim()[0], right=0.5)  # Extends the right limit to 0.5 seconds

    # Set explicit x-axis ticks to ensure 0.5 is included and labeled
    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Set major ticks interval
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))  # Set minor ticks interval
    # Adjust font size of axis labels and tick labels
    ax.set_xlabel('Time (s)', fontsize=14)  # Time in seconds with larger font
    ax.set_ylabel('Amplitude (µV)', fontsize=14)  # Amplitude in microvolts with larger font
    ax.tick_params(axis='both', which='major', labelsize=12)  # Tick labels font size for major ticks
    ax.tick_params(axis='both', which='minor', labelsize=10)  # Tick labels font size for minor ticks
    # Adjust the title font size
    ax.set_title(f"Participant '{participant}' ERPs", fontsize=16)  
    # Force a redraw of the figure to ensure the plot updates
    plt.draw()

    # Optionally, add grid for better readability
    ax.grid(True)

    # Optionally, set labels for the axes to indicate units
    ax.set_xlabel('Time (s)')  # Time in seconds
    ax.set_ylabel('Amplitude (µV)')  # Amplitude in microvolts
    plt.show()

colors = {'US': 'red', 'AS': 'blue', 'UR': 'red', 'AR': 'blue'}  

for participant, evokeds in all_parts_combined.items():
    if evokeds:  # Only plot if there are evoked data present
        plot_evokeds_for_participant(participant, evokeds, colors)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting 

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting 

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting 

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting 

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  ev.pick_channels(list(common_channels))
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1965532854.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting 

In [49]:
"""ERP of grand mean of all participants"""

combined_evokeds_by_cond = {cond: [] for cond in conds_of_interest}
participant_count_by_cond = {}  # Dictionary to hold participant counts for each condition

for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        cond = simplify_condition(evoked.comment)
        if cond:
            combined_evokeds_by_cond[cond].append(evoked)
            if cond not in participant_count_by_cond:
                participant_count_by_cond[cond] = set()
            participant_count_by_cond[cond].add(participant)

# Adjusting each evoked data to only include common channels
all_channels = set.intersection(*(set(evoked.ch_names) for evokeds in combined_evokeds_by_cond.values() for evoked in evokeds))
for cond, evokeds in combined_evokeds_by_cond.items():
    for evoked in evokeds:
        evoked.pick_channels(list(all_channels))  # Updated method to use pick_channels

grand_averages_all = []
legend_info = {}
for cond, evokeds in combined_evokeds_by_cond.items():
    if evokeds:
        grand_avg = mne.grand_average(evokeds)
        grand_avg.comment = cond
        grand_averages_all.append(grand_avg)
        # Prepare legend info with participant count
        legend_info[cond] = f"{cond} (n={len(participant_count_by_cond[cond])})"

# Dictionary for plot styling
colors = {
    'US': 'red',
    'UR': 'red',
    'AS': 'blue',
    'AR': 'blue'
}
styles = {
    'UR': {'linestyle': '--'},
    'AR': {'linestyle': '--'}
}

# Update styles and colors to use the legend_info keys
updated_colors = {legend_info[cond]: colors[cond] for cond in colors if cond in legend_info}
updated_styles = {legend_info[cond]: styles[cond] for cond in styles if cond in legend_info}

# Function to add shaded regions with labels
def add_shaded_region_with_label(ax, start, end, label, y_max):
    ax.axvspan(start, end, color='gray', alpha=0.3)
    midpoint = (start + end) / 2
    ax.text(midpoint, y_max * 0.9, label, horizontalalignment='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

# Plotting
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(
    {legend_info[cond]: grand_averages_all[i] for i, cond in enumerate(conds_of_interest) if cond in legend_info},
    picks="Cz",
    ci=0.95,
    colors=updated_colors,
    styles=updated_styles,
    title="Grand Average Evoked Responses by Condition",
    axes=ax
)

# Get y-axis limits for label placement
y_min, y_max = ax.get_ylim()

# Add shaded regions and labels
add_shaded_region_with_label(ax, 0.19, 0.3, 'P200', y_max)
add_shaded_region_with_label(ax, 0.305, 0.5, 'P400', y_max)

# Customizing the legend manually
handles, labels = ax.get_legend_handles_labels()
new_labels = [legend_info[label.split()[0]] for label in labels]  # Remapping labels to include participant counts
ax.legend(handles, new_labels, fontsize=15)

# Extending the x-axis to show time until 0.5 seconds
ax.set_xlim(left=ax.get_xlim()[0], right=0.5)  # Extends the right limit to 0.5 seconds

# Set explicit x-axis ticks to ensure 0.5 is included and labeled
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Set major ticks interval
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))  # Set minor ticks interval
# Adjust font size of axis labels and tick labels
ax.set_xlabel('Time (s)', fontsize=14)  # Time in seconds with larger font
ax.set_ylabel('Amplitude (µV)', fontsize=14)  # Amplitude in microvolts with larger font
ax.tick_params(axis='both', which='major', labelsize=12)  # Tick labels font size for major ticks
ax.tick_params(axis='both', which='minor', labelsize=10)  # Tick labels font size for minor ticks
# Force a redraw of the figure to ensure the plot updates
plt.draw()

# Optionally, add grid for better readability
ax.grid(True)

# Ensure everything fits in the plot area
plt.tight_layout()
# Add figure name below the x-axis label
fig.text(0.5, 0.001, 'Plot A', ha='center', fontsize=14)

plt.show()


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3441582458.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  evoked.pick_channels(list(all_channels))  # Updated method to use pick_channels
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3441582458.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  evoked.pick_channels(list(all_channels))  # Updated method to use pick_channels
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3441582458.py:19: FutureWarning: The default for pick_channels will change from ordered=False to ordered

In [10]:
"""Voltage maps for P200 and P400"""

# Define the time windows for P200 and P400
p200_window = (0.19, 0.3)
p400_window = (0.305, 0.5)

# Create voltage maps for P200 and P400
for grand_avg in grand_averages_all:
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    # P200 component
    p200_data = grand_avg.copy().crop(tmin=p200_window[0], tmax=p200_window[1]).data.mean(axis=1)
    mne.viz.plot_topomap(p200_data, grand_avg.info, axes=axes[0], show=False)
    axes[0].set_title(f'{grand_avg.comment} - P200 (190-300 ms)')

    # P400 component
    p400_data = grand_avg.copy().crop(tmin=p400_window[0], tmax=p400_window[1]).data.mean(axis=1)
    mne.viz.plot_topomap(p400_data, grand_avg.info, axes=axes[1], show=False)
    axes[1].set_title(f'{grand_avg.comment} - P400 (305-500 ms)')

    # Add a colorbar
    plt.colorbar(axes[0].images[0], ax=axes, orientation='horizontal')
    plt.suptitle(f'Voltage Maps for {grand_avg.comment}', fontsize=16)
    plt.tight_layout()
    plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3106993419.py:24: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3106993419.py:24: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3106993419.py:24: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3106993419.py:24: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [48]:
"""ERP of grand mean of all participants with confidence interval"""

combined_evokeds_by_cond = {cond: [] for cond in conds_of_interest}
participant_count_by_cond = {}  # Dictionary to hold participant counts for each condition

for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        cond = simplify_condition(evoked.comment)
        if cond:
            combined_evokeds_by_cond[cond].append(evoked)
            if cond not in participant_count_by_cond:
                participant_count_by_cond[cond] = set()
            participant_count_by_cond[cond].add(participant)

# Dictionary for plot styling
colors = {
    'US': 'red',
    'UR': 'red',
    'AS': 'blue',
    'AR': 'blue'
}
styles = {
    'UR': {'linestyle': '--'},
    'AR': {'linestyle': '--'}
}

# Prepare the evoked data with participant counts for plotting
plot_evokeds = {}
for cond in combined_evokeds_by_cond:
    if combined_evokeds_by_cond[cond]:
        plot_evokeds[f"{cond} (n={len(participant_count_by_cond[cond])})"] = combined_evokeds_by_cond[cond]

# Update colors and styles to use the condition names with participant counts
updated_colors = {f"{cond} (n={len(participant_count_by_cond[cond])})": colors[cond] for cond in colors}
updated_styles = {f"{cond} (n={len(participant_count_by_cond[cond])})": styles[cond] for cond in styles if cond in styles}

# Plotting with CIs
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(
    plot_evokeds,
    picks="Cz",
    ci=0.95,
    colors=updated_colors,
    styles=updated_styles,
    title="Grand Average Evoked Responses by Condition",
    axes=ax
)

# Customizing the legend manually
handles, labels = ax.get_legend_handles_labels()
new_labels = [legend_info[label.split()[0]] for label in labels]  # Remapping labels to include participant counts
ax.legend(handles, new_labels, fontsize=14)

# Extending the x-axis to show time until 0.5 seconds
ax.set_xlim(left=ax.get_xlim()[0], right=0.5)  # Extends the right limit to 0.5 seconds

# Set explicit x-axis ticks to ensure 0.5 is included and labeled
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Set major ticks interval
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))  # Set minor ticks interval
# Adjust font size of axis labels and tick labels
ax.set_xlabel('Time (s)', fontsize=14)  # Time in seconds with larger font
ax.set_ylabel('Amplitude (µV)', fontsize=14)  # Amplitude in microvolts with larger font
ax.tick_params(axis='both', which='major', labelsize=12)  # Tick labels font size for major ticks
ax.tick_params(axis='both', which='minor', labelsize=10)  # Tick labels font size for minor ticks

# Adjust the title font size
ax.set_title("Grand Average ERPs by Condition", fontsize=16)  # Title with larger font

# Adjust layout to make room for the figure name
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# Add figure name below the x-axis label
fig.text(0.5, 0.01, 'Figure A', ha='center', fontsize=14)

# Force a redraw of the figure to ensure the plot updates
plt.draw()

# Optionally, add grid for better readability
ax.grid(True)

# Optionally, set labels for the axes to indicate units
ax.set_xlabel('Time (s)')  # Time in seconds
ax.set_ylabel('Amplitude (µV)')  # Amplitude in microvolts

plt.show()


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1079379902.py:23: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  evoked.pick_channels(list(all_channels))  # Updated method to use pick_channels
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1079379902.py:23: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  evoked.pick_channels(list(all_channels))  # Updated method to use pick_channels
C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1079379902.py:23: FutureWarning: The default for pick_channels will change from ordered=False to ordered

Identifying common channels ...
Identifying common channels ...


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1079379902.py:100: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [29]:

"""ERP of grand mean of all participants with confidence interval ambiguous and unambiguous conditions seperately"""
# Separate conditions into ambiguous and unambiguous
ambiguous_conds = ['AS', 'AR']
unambiguous_conds = ['US', 'UR']

# Create dictionaries to store combined evoked data for each set
combined_evokeds_by_ambiguous = {cond: [] for cond in ambiguous_conds}
combined_evokeds_by_unambiguous = {cond: [] for cond in unambiguous_conds}

# Participant count dictionaries
participant_count_by_ambiguous = {}
participant_count_by_unambiguous = {}

# Separate evoked data into ambiguous and unambiguous
for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        cond = simplify_condition(evoked.comment)
        if cond in ambiguous_conds:
            combined_evokeds_by_ambiguous[cond].append(evoked)
            if cond not in participant_count_by_ambiguous:
                participant_count_by_ambiguous[cond] = set()
            participant_count_by_ambiguous[cond].add(participant)
        elif cond in unambiguous_conds:
            combined_evokeds_by_unambiguous[cond].append(evoked)
            if cond not in participant_count_by_unambiguous:
                participant_count_by_unambiguous[cond] = set()
            participant_count_by_unambiguous[cond].add(participant)

# Prepare evoked data with participant counts for plotting ambiguous conditions
plot_evokeds_ambiguous = {}
for cond in combined_evokeds_by_ambiguous:
    if combined_evokeds_by_ambiguous[cond]:
        plot_evokeds_ambiguous[f"{cond}\n(n={len(participant_count_by_ambiguous[cond])})"] = combined_evokeds_by_ambiguous[cond]

# Prepare evoked data with participant counts for plotting unambiguous conditions
plot_evokeds_unambiguous = {}
for cond in combined_evokeds_by_unambiguous:
    if combined_evokeds_by_unambiguous[cond]:
        plot_evokeds_unambiguous[f"{cond}\n(n={len(participant_count_by_unambiguous[cond])})"] = combined_evokeds_by_unambiguous[cond]

# Update colors and styles
colors = {
    'US': 'darkred',
    'UR': 'red',
    'AS': 'darkblue',
    'AR': 'blue'
}
styles = {
    'UR': {'linestyle': '--'},
    'AR': {'linestyle': '--'}
}

updated_colors_ambiguous = {f"{cond}\n(n={len(participant_count_by_ambiguous[cond])})": colors[cond] for cond in ambiguous_conds}
updated_styles_ambiguous = {f"{cond}\n(n={len(participant_count_by_ambiguous[cond])})": styles[cond] for cond in styles if cond in ambiguous_conds}

updated_colors_unambiguous = {f"{cond}\n(n={len(participant_count_by_unambiguous[cond])})": colors[cond] for cond in unambiguous_conds}
updated_styles_unambiguous = {f"{cond}\n(n={len(participant_count_by_unambiguous[cond])})": styles[cond] for cond in styles if cond in unambiguous_conds}

# Function to add shaded regions with labels
def add_shaded_region_with_label(ax, start, end, label, y_max):
    ax.axvspan(start, end, color='gray', alpha=0.3)
    midpoint = (start + end) / 2
    ax.text(midpoint, y_max * 0.9, label, horizontalalignment='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

# Plotting ambiguous conditions
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(
    plot_evokeds_ambiguous,
    picks="Cz",
    ci=0.95,
    colors=updated_colors_ambiguous,
    styles=updated_styles_ambiguous,
    title="Grand Average ERPs - Ambiguos",
    axes=ax
)

# Get y-axis limits for label placement
y_min, y_max = ax.get_ylim()

# Add shaded regions and labels
add_shaded_region_with_label(ax, 0.19, 0.3, 'P200', y_max)
add_shaded_region_with_label(ax, 0.305, 0.5, 'P400', y_max)

# Customizing the legend manually
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, fontsize=12, loc='upper left')  # Adjust legend font size here

# Extending the x-axis to show time until 0.5 seconds
ax.set_xlim(left=ax.get_xlim()[0], right=0.5)  # Extends the right limit to 0.5 seconds

# Set explicit x-axis ticks to ensure 0.5 is included and labeled
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Set major ticks interval
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))  # Set minor ticks interval

# Adjust font size of axis labels and tick labels
ax.set_xlabel('Time (s)', fontsize=14)  # Time in seconds with larger font
ax.set_ylabel('Amplitude (µV)', fontsize=14)  # Amplitude in microvolts with larger font
ax.tick_params(axis='both', which='major', labelsize=12)  # Tick labels font size for major ticks
ax.tick_params(axis='both', which='minor', labelsize=10)  # Tick labels font size for minor ticks

# Adjust the title font size
ax.set_title("Grand Average ERPs - Ambiguous", fontsize=16)  # Title with larger font

# Set consistent Y-axis ticks
ax.set_yticks([-10, -7.5, -5, -2.5, 0, 2.5, 5, 7.5, 10])
# Add figure name
fig.text(0.5, 0.01, 'Plot A', ha='center', fontsize=14)

# Force a redraw of the figure to ensure the plot updates
plt.draw()

# Optionally, add grid for better readability
ax.grid(True)

# Ensure everything fits in the plot area
plt.tight_layout()

plt.show()

# Plotting unambiguous conditions
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(
    plot_evokeds_unambiguous,
    picks="Cz",
    ci=0.95,
    colors=updated_colors_unambiguous,
    styles=updated_styles_unambiguous,
    title="Grand Average ERPs - Unambiguous",
    axes=ax
)

# Get y-axis limits for label placement
y_min, y_max = ax.get_ylim()

# Add shaded regions and labels
add_shaded_region_with_label(ax, 0.19, 0.3, 'P200', y_max)
add_shaded_region_with_label(ax, 0.305, 0.5, 'P400', y_max)

# Customizing the legend manually
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, fontsize=12, loc='upper left')  # Adjust legend font size here

# Extending the x-axis to show time until 0.5 seconds
ax.set_xlim(left=ax.get_xlim()[0], right=0.5)  # Extends the right limit to 0.5 seconds

# Set explicit x-axis ticks to ensure 0.5 is included and labeled
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Set major ticks interval
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))  # Set minor ticks interval

# Adjust font size of axis labels and tick labels
ax.set_xlabel('Time (s)', fontsize=14)  # Time in seconds with larger font
ax.set_ylabel('Amplitude (µV)', fontsize=14)  # Amplitude in microvolts with larger font
ax.tick_params(axis='both', which='major', labelsize=12)  # Tick labels font size for major ticks
ax.tick_params(axis='both', which='minor', labelsize=10)  # Tick labels font size for minor ticks

# Adjust the title font size
ax.set_title("Grand Average ERPs - Unambiguous", fontsize=16)  # Title with larger font

# Set consistent Y-axis ticks
ax.set_yticks([-10, -7.5, -5, -2.5, 0, 2.5, 5, 7.5, 10])
# Add figure name
fig.text(0.5, 0.01, 'Plot B', ha='center', fontsize=14)

# Force a redraw of the figure to ensure the plot updates
plt.draw()

# Optionally, add grid for better readability
ax.grid(True)

# Ensure everything fits in the plot area
plt.tight_layout()

plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\1644472798.py:116: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\1644472798.py:171: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [30]:
"""ERP of grand mean of all participants with confidence interval steady state and random conditions seperately"""

# Separate conditions into steady state and random
steady_state_conds = ['US', 'AS']
random_conds = ['UR', 'AR']

# Create dictionaries to store combined evoked data for each set
combined_evokeds_by_steady_state = {cond: [] for cond in steady_state_conds}
combined_evokeds_by_random = {cond: [] for cond in random_conds}

# Participant count dictionaries
participant_count_by_steady_state = {}
participant_count_by_random = {}

# Separate evoked data into steady state and random
for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        cond = simplify_condition(evoked.comment)
        if cond in steady_state_conds:
            combined_evokeds_by_steady_state[cond].append(evoked)
            if cond not in participant_count_by_steady_state:
                participant_count_by_steady_state[cond] = set()
            participant_count_by_steady_state[cond].add(participant)
        elif cond in random_conds:
            combined_evokeds_by_random[cond].append(evoked)
            if cond not in participant_count_by_random:
                participant_count_by_random[cond] = set()
            participant_count_by_random[cond].add(participant)

# Prepare evoked data with participant counts for plotting steady state conditions
plot_evokeds_steady_state = {}
for cond in combined_evokeds_by_steady_state:
    if combined_evokeds_by_steady_state[cond]:
        plot_evokeds_steady_state[f"{cond}\n(n={len(participant_count_by_steady_state[cond])})"] = combined_evokeds_by_steady_state[cond]

# Prepare evoked data with participant counts for plotting random conditions
plot_evokeds_random = {}
for cond in combined_evokeds_by_random:
    if combined_evokeds_by_random[cond]:
        plot_evokeds_random[f"{cond}\n(n={len(participant_count_by_random[cond])})"] = combined_evokeds_by_random[cond]

# Update colors and styles
colors = {
    'US': 'darkred',
    'UR': 'red',
    'AS': 'darkblue',
    'AR': 'blue'
}
styles = {
    'UR': {'linestyle': '--'},
    'AR': {'linestyle': '--'}
}

updated_colors_steady_state = {f"{cond}\n(n={len(participant_count_by_steady_state[cond])})": colors[cond] for cond in steady_state_conds}
updated_styles_steady_state = {f"{cond}\n(n={len(participant_count_by_steady_state[cond])})": styles[cond] for cond in styles if cond in steady_state_conds}

updated_colors_random = {f"{cond}\n(n={len(participant_count_by_random[cond])})": colors[cond] for cond in random_conds}
updated_styles_random = {f"{cond}\n(n={len(participant_count_by_random[cond])})": styles[cond] for cond in styles if cond in random_conds}

# Function to add shaded regions with labels
def add_shaded_region_with_label(ax, start, end, label, y_max):
    ax.axvspan(start, end, color='gray', alpha=0.3)
    midpoint = (start + end) / 2
    ax.text(midpoint, y_max * 0.9, label, horizontalalignment='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

# Plotting steady state conditions
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(
    plot_evokeds_steady_state,
    picks="Cz",
    ci=0.95,
    colors=updated_colors_steady_state,
    styles=updated_styles_steady_state,
    title="Grand Average ERPs - Steady State",
    axes=ax
)

# Get y-axis limits for label placement
y_min, y_max = ax.get_ylim()

# Add shaded regions and labels
add_shaded_region_with_label(ax, 0.19, 0.3, 'P200', y_max)
add_shaded_region_with_label(ax, 0.305, 0.5, 'P400', y_max)

# Customizing the legend manually
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, fontsize=12, loc='upper left')  # Adjust legend font size here

# Extending the x-axis to show time until 0.5 seconds
ax.set_xlim(left=ax.get_xlim()[0], right=0.5)  # Extends the right limit to 0.5 seconds

# Set explicit x-axis ticks to ensure 0.5 is included and labeled
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Set major ticks interval
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))  # Set minor ticks interval

# Adjust font size of axis labels and tick labels
ax.set_xlabel('Time (s)', fontsize=14)  # Time in seconds with larger font
ax.set_ylabel('Amplitude (µV)', fontsize=14)  # Amplitude in microvolts with larger font
ax.tick_params(axis='both', which='major', labelsize=12)  # Tick labels font size for major ticks
ax.tick_params(axis='both', which='minor', labelsize=10)  # Tick labels font size for minor ticks

# Adjust the title font size
ax.set_title("Grand Average ERPs - Steady State", fontsize=16)  # Title with larger font

# Set consistent Y-axis ticks
ax.set_yticks([-10, -7.5, -5, -2.5, 0, 2.5, 5, 7.5, 10])

# Add figure name
fig.text(0.5, 0.01, 'Plot B', ha='center', fontsize=14)

# Force a redraw of the figure to ensure the plot updates
plt.draw()

# Optionally, add grid for better readability
ax.grid(True)

# Ensure everything fits in the plot area
plt.tight_layout()

plt.show()

# Plotting random conditions
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(
    plot_evokeds_random,
    picks="Cz",
    ci=0.95,
    colors=updated_colors_random,
    styles=updated_styles_random,
    title="Grand Average ERPs - Random",
    axes=ax
)

# Get y-axis limits for label placement
y_min, y_max = ax.get_ylim()

# Add shaded regions and labels
add_shaded_region_with_label(ax, 0.19, 0.3, 'P200', y_max)
add_shaded_region_with_label(ax, 0.305, 0.5, 'P400', y_max)

# Customizing the legend manually
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, fontsize=12, loc='upper left')  # Adjust legend font size here

# Extending the x-axis to show time until 0.5 seconds
ax.set_xlim(left=ax.get_xlim()[0], right=0.5)  # Extends the right limit to 0.5 seconds

# Set explicit x-axis ticks to ensure 0.5 is included and labeled
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))  # Set major ticks interval
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))  # Set minor ticks interval

# Adjust font size of axis labels and tick labels
ax.set_xlabel('Time (s)', fontsize=14)  # Time in seconds with larger font
ax.set_ylabel('Amplitude (µV)', fontsize=14)  # Amplitude in microvolts with larger font
ax.tick_params(axis='both', which='major', labelsize=12)  # Tick labels font size for major ticks
ax.tick_params(axis='both', which='minor', labelsize=10)  # Tick labels font size for minor ticks

# Adjust the title font size
ax.set_title("Grand Average ERPs - Random", fontsize=16)  # Title with larger font

# Set consistent Y-axis ticks
ax.set_yticks([-10, -7.5, -5, -2.5, 0, 2.5, 5, 7.5, 10])

# Add figure name
fig.text(0.5, 0.01, 'Plot C', ha='center', fontsize=14)

# Force a redraw of the figure to ensure the plot updates
plt.draw()

# Optionally, add grid for better readability
ax.grid(True)

# Ensure everything fits in the plot area
plt.tight_layout()

plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\1087814396.py:118: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\1087814396.py:174: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [35]:

# Increase the maximum number of rows to display the entire DataFrame
pd.set_option('display.max_rows', None)
data = []
for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        data.append({
            "Participant": participant,
            "Condition": evoked.comment.split('/')[0],  
            "N": evoked.nave
        })

df_evokeds = pd.DataFrame(data)
# print(df_evokeds)

# Display the DataFrame of all participants after preprocessing
display(df_evokeds)




,Participant,Condition,N
0,BA,US,151
1,BA,AS,145
2,BA,UR,130
3,BA,AR,152
4,BC,US,89
5,BC,AS,96
6,BC,UR,150
7,BC,AR,152
8,BD,US,151
9,BD,AS,152


In [12]:
"""Finding peak function for P200 and P400"""

# Function to detect peaks in the evoked response
def detect_peaks(evoked, ch_name='Cz'):
    sfreq = evoked.info['sfreq']  # Sample frequency

    # Time ranges for P200 and P400 in seconds
    P200_range = (0.29, 0.4)
    P400_range = (0.4, 0.6)
    # Convert time ranges to sample indices
    idx_range_P2 = (int(P200_range[0] * sfreq), int(P200_range[1] * sfreq))
    idx_range_P4 = (int(P400_range[0] * sfreq), int(P400_range[1] * sfreq))
    
    # Select data for 'Cz' electrode
    ch_idx = mne.pick_channels(evoked.ch_names, include=[ch_name])
    data_P2 = evoked.data[ch_idx, idx_range_P2[0]:idx_range_P2[1]][0]
    data_P4 = evoked.data[ch_idx, idx_range_P4[0]:idx_range_P4[1]][0]

    # Detect peaks in the specified ranges
    peaks_P2 = find_peaks(data_P2, distance=sfreq*P200_range[0])[0]  # enforcing minimum distance
    peaks_P4 = find_peaks(data_P4, distance=sfreq*P400_range[0])[0]

    # Find peak with the maximum amplitude in each range
    if len(peaks_P2) > 0:
        peak_amp_P2 = data_P2[peaks_P2[np.argmax(data_P2[peaks_P2])]]
        peak_time_P2 = evoked.times[idx_range_P2[0] + peaks_P2[np.argmax(data_P2[peaks_P2])]]
    else:
        peak_amp_P2 = None
        peak_time_P2 = None

    if len(peaks_P4) > 0:
        peak_amp_P4 = data_P4[peaks_P4[np.argmax(data_P4[peaks_P4])]]
        peak_time_P4 = evoked.times[idx_range_P4[0] + peaks_P4[np.argmax(data_P4[peaks_P4])]]
    else:
        peak_amp_P4 = None
        peak_time_P4 = None

    return {
        'P200_amplitude': peak_amp_P2,
        'P200_time': peak_time_P2,
        'P400_amplitude': peak_amp_P4,
        'P400_time': peak_time_P4
    }


In [11]:
"""Plot all peaks from participants and save it"""
## Find peaks for all participants
for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        # Detect peaks (you should have this function from previous snippets)
        peaks = detect_peaks(evoked)
        print(f"Condition {evoked.comment} - P200 Time: {peaks['P200_time']}, Amplitude: {peaks['P200_amplitude']}")
        print("-----------------------------------------------------------------------------------------------------")
        print(f"Condition {evoked.comment} - P400 Time: {peaks['P400_time']}, Amplitude: {peaks['P400_amplitude']}")
        if peaks['P400_time'] is not None:
            print(f"Detected P400 peak at {peaks['P400_time']} s")
        
        lat_P2 = peaks.get('P200_time', None)
        lat_P4 = peaks.get('P400_time', None)
        # Prepare the figure
        fig = plt.figure(figsize=(8, 10))
        ax1 = fig.add_subplot(2, 1, 1)
        ax2 = fig.add_subplot(2, 2, 3)
        ax3 = fig.add_subplot(2, 2, 4)

        # Plot the compare evokeds for 'Cz' with vertical lines at the peak latencies
        mne.viz.plot_compare_evokeds(
            evoked.copy().pick(['Cz']), 
            vlines=[lat_P2, lat_P4] if None not in [lat_P2, lat_P4] else None,
            legend=False, 
            title=f'Participant {participant} - Condition {evoked.comment}', 
            axes=ax1
        )

        # Plot topomaps at the detected peak times
        if lat_P2 is not None:
            evoked.plot_topomap(times=[lat_P2], ch_type="eeg", colorbar=False, axes=ax2)
        if lat_P4 is not None:
            evoked.plot_topomap(times=[lat_P4], ch_type="eeg", colorbar=False, axes=ax3)

        # Adjust layout and save the figure
        fig.tight_layout()
        filename = f'Participant_{participant}_{evoked.comment}_peaks'
        # fig.savefig(f'{filename}.png') ## save the file in the directory
        plt.show()
# if 'AO' in all_parts:
#     evokeds = all_parts['AO']
    # for evoked in evokeds:
    #         # Detect peaks (you should have this function from previous snippets)
    #         peaks = detect_peaks(evoked)
    #         print(f"Condition {evoked.comment} - P200 Time: {peaks['P200_time']}, Amplitude: {peaks['P200_amplitude']}")
    #         print(f"Condition {evoked.comment} - P400 Time: {peaks['P400_time']}, Amplitude: {peaks['P400_amplitude']}")
    #         if peaks['P400_time'] is not None:
    #             print(f"Detected P400 peak at {peaks['P400_time']} s")
            
    #         lat_P2 = peaks.get('P200_time', None)
    #         lat_P4 = peaks.get('P400_time', None)
    #         # Prepare the figure
    #         fig = plt.figure(figsize=(8, 10))
    #         ax1 = fig.add_subplot(2, 1, 1)
    #         ax2 = fig.add_subplot(2, 2, 3)
    #         ax3 = fig.add_subplot(2, 2, 4)

    #         # Plot the compare evokeds for 'Cz' with vertical lines at the peak latencies
    #         mne.viz.plot_compare_evokeds(
    #             evoked.copy().pick(['Cz']), 
    #             vlines=[lat_P2, lat_P4] if None not in [lat_P2, lat_P4] else None,
    #             legend=False, 
    #             title=f'Participant {participant} - Condition {evoked.comment}', 
    #             axes=ax1
    #         )

    #         # Plot topomaps at the detected peak times
    #         if lat_P2 is not None:
    #             evoked.plot_topomap(times=[lat_P2], ch_type="eeg", colorbar=False, axes=ax2)
    #         if lat_P4 is not None:
    #             evoked.plot_topomap(times=[lat_P4], ch_type="eeg", colorbar=False, axes=ax3)

    #         # Adjust layout and save the figure
    #         fig.tight_layout()
    #         # filename = f'Participant_{participant}_{evoked.comment}_peaks'
    #         # fig.savefig(f'{filename}.png')
    #         plt.show()
else:
    print("Participant {participant} not found in the dataset.")


NameError: name 'detect_peaks' is not defined

In [45]:
"""UNCORRECTED ANOVA TEST FOR ALL PARTICIPANTS"""
# Create an empty DataFrame
df2122 = pd.DataFrame(columns=['Participant', 'Condition', 'Ambiguity', 'Randomness', 'P200_Amplitude', 'P400_Amplitude'])

# Populate the DataFrame with the peak detection results
for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        peaks = detect_peaks(evoked)
        entry = pd.DataFrame({
            'Participant': [participant],
            'Condition': [evoked.comment],
            'Ambiguity': ['Ambiguous' if 'A' in evoked.comment else 'Unambiguous'],
            'Randomness': ['Random' if 'R' in evoked.comment else 'Steady State'],
            'P200_Amplitude': [peaks.get('P200_amplitude', np.nan)],  # use np.nan for missing data
            'P200_Latency': peaks.get('P200_time', np.nan),
            'P400_Amplitude': [peaks.get('P400_amplitude', np.nan)],
            'P400_Latency': [peaks.get('P400_time', np.nan)]
        })
        
        df2122 = pd.concat([df2122, entry], ignore_index=True)
# print(df2122)
df2122['P200_Amplitude'] = df2122['P200_Amplitude'] * 1e6 ### convert to microvolt
df2122['P400_Amplitude'] = df2122['P400_Amplitude'] * 1e6
# Running ANOVA for P200 amplitude
model_p200_amp = AnovaRM(df2122.dropna(subset=['P200_Amplitude']), 'P200_Amplitude', 'Participant', within=['Ambiguity', 'Randomness'])
res_p200_amp = model_p200_amp.fit()
print("P200 Amplitude ANOVA")
print(res_p200_amp)

# Running ANOVA for P400 amplitude
model_p400_amp = AnovaRM(df2122.dropna(subset=['P400_Amplitude']), 'P400_Amplitude', 'Participant', within=['Ambiguity', 'Randomness'])
res_p400_amp = model_p400_amp.fit()
print("P400 Amplitude ANOVA")
print(res_p400_amp)

# Running ANOVA for P200 latency
model_p200_lat = AnovaRM(df2122.dropna(subset=['P200_Latency']), 'P200_Latency', 'Participant', within=['Ambiguity', 'Randomness'])
res_p200_lat = model_p200_lat.fit()
print("P200 Latency ANOVA")
print(res_p200_lat)

# Running ANOVA for P400 latency
model_p400_lat = AnovaRM(df2122.dropna(subset=['P400_Latency']), 'P400_Latency', 'Participant', within=['Ambiguity', 'Randomness'])
res_p400_lat = model_p400_lat.fit()
print("P400 Latency ANOVA")
print(res_p400_lat)


P200 Amplitude ANOVA
                      Anova
                     F Value Num DF  Den DF Pr > F
--------------------------------------------------
Ambiguity             1.3298 1.0000 22.0000 0.2612
Randomness            1.3794 1.0000 22.0000 0.2528
Ambiguity:Randomness  0.6274 1.0000 22.0000 0.4368

P400 Amplitude ANOVA
                      Anova
                     F Value Num DF  Den DF Pr > F
--------------------------------------------------
Ambiguity            40.2018 1.0000 22.0000 0.0000
Randomness            5.3990 1.0000 22.0000 0.0298
Ambiguity:Randomness  0.7003 1.0000 22.0000 0.4117

P200 Latency ANOVA
                      Anova
                     F Value Num DF  Den DF Pr > F
--------------------------------------------------
Ambiguity             2.0781 1.0000 22.0000 0.1635
Randomness           12.8186 1.0000 22.0000 0.0017
Ambiguity:Randomness  0.3489 1.0000 22.0000 0.5608

P400 Latency ANOVA
                      Anova
                     F Value Num DF  Den

C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\875302821.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2122 = pd.concat([df2122, entry], ignore_index=True)


In [43]:
"""Corrected Anova test"""

num_tests = 4
# Run ANOVA for P200 Amplitude, P400 Amplitude, P200 Latency, and P400 Latency
res_p200_amp = AnovaRM(df2122.dropna(subset=['P200_Amplitude']), 'P200_Amplitude', 'Participant', within=['Ambiguity', 'Randomness']).fit()
res_p400_amp = AnovaRM(df2122.dropna(subset=['P400_Amplitude']), 'P400_Amplitude', 'Participant', within=['Ambiguity', 'Randomness']).fit()
res_p200_lat = AnovaRM(df2122.dropna(subset=['P200_Latency']), 'P200_Latency', 'Participant', within=['Ambiguity', 'Randomness']).fit()
res_p400_lat = AnovaRM(df2122.dropna(subset=['P400_Latency']), 'P400_Latency', 'Participant', within=['Ambiguity', 'Randomness']).fit()

# Apply Bonferroni correction
corrected_p200_amp = res_p200_amp.anova_table['Pr > F'] * num_tests
corrected_p400_amp = res_p400_amp.anova_table['Pr > F'] * num_tests
corrected_p200_lat = res_p200_lat.anova_table['Pr > F'] * num_tests
corrected_p400_lat = res_p400_lat.anova_table['Pr > F'] * num_tests

# Ensure p-values do not exceed 1 after correction
corrected_p200_amp = corrected_p200_amp.apply(lambda x: min(x, 1))
corrected_p400_amp = corrected_p400_amp.apply(lambda x: min(x, 1))
corrected_p200_lat = corrected_p200_lat.apply(lambda x: min(x, 1))
corrected_p400_lat = corrected_p400_lat.apply(lambda x: min(x, 1))

# Printing results
print("P200 Amplitude ANOVA")
print(res_p200_amp)
print("\nBonferroni Corrected P200 Amplitude ANOVA p-values:")
print(corrected_p200_amp)

print("\nP400 Amplitude ANOVA")
print(res_p400_amp)
print("\nBonferroni Corrected P400 Amplitude ANOVA p-values:")
print(corrected_p400_amp)

print("\nP200 Latency ANOVA")
print(res_p200_lat)
print("\nBonferroni Corrected P200 Latency ANOVA p-values:")
print(corrected_p200_lat)

print("\nP400 Latency ANOVA")
print(res_p400_lat)
print("\nBonferroni Corrected P400 Latency ANOVA p-values:")
print(corrected_p400_lat)

P200 Amplitude ANOVA
                      Anova
                     F Value Num DF  Den DF Pr > F
--------------------------------------------------
Ambiguity             1.3298 1.0000 22.0000 0.2612
Randomness            1.3794 1.0000 22.0000 0.2528
Ambiguity:Randomness  0.6274 1.0000 22.0000 0.4368


Bonferroni Corrected P200 Amplitude ANOVA p-values:
Ambiguity               1
Randomness              1
Ambiguity:Randomness    1
Name: Pr > F, dtype: int64

P400 Amplitude ANOVA
                      Anova
                     F Value Num DF  Den DF Pr > F
--------------------------------------------------
Ambiguity            40.2018 1.0000 22.0000 0.0000
Randomness            5.3990 1.0000 22.0000 0.0298
Ambiguity:Randomness  0.7003 1.0000 22.0000 0.4117


Bonferroni Corrected P400 Amplitude ANOVA p-values:
Ambiguity               0.000009
Randomness              0.119135
Ambiguity:Randomness    1.000000
Name: Pr > F, dtype: float64

P200 Latency ANOVA
                      Anova
  

In [44]:
"""Uncorrected Anova test with details for all participants"""

# Create an empty DataFrame
dfAll = pd.DataFrame(columns=['Participant', 'Condition', 'Ambiguity', 'Randomness', 'P200_Amplitude', 'P200_Latency', 'P400_Amplitude', 'P400_Latency'])

# Populate the DataFrame with the peak detection results
for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        peaks = detect_peaks(evoked)
        entry = pd.DataFrame({
            'Participant': [participant],
            'Condition': [evoked.comment],
            'Ambiguity': ['Ambiguous' if 'A' in evoked.comment else 'Unambiguous'],
            'Randomness': ['Random' if 'R' in evoked.comment else 'Steady State'],
            'P200_Amplitude': [peaks.get('P200_amplitude', np.nan)],  # use np.nan for missing data
            'P200_Latency': [peaks.get('P200_time', np.nan)],
            'P400_Amplitude': [peaks.get('P400_amplitude', np.nan)],
            'P400_Latency': [peaks.get('P400_time', np.nan)] 
        })
        
        dfAll = pd.concat([dfAll, entry], ignore_index=True)

dfAll['P200_Amplitude'] = dfAll['P200_Amplitude'] * 1e6
dfAll['P400_Amplitude'] = dfAll['P400_Amplitude'] * 1e6

def calculate_partial_eta_squared(f_value, df_effect, df_error):
    return (f_value * df_effect) / (f_value * df_effect + df_error)

# Function to print ANOVA results with effect sizes
def print_anova_with_effect_sizes(res):
    anova_table = res.anova_table
    for effect in anova_table.index:
        f_value = anova_table.loc[effect, 'F Value']
        df_effect = anova_table.loc[effect, 'Num DF']
        df_error = anova_table.loc[effect, 'Den DF']
        partial_eta_squared = calculate_partial_eta_squared(f_value, df_effect, df_error)
        print(f"{effect} - F Value: {f_value}, p-value: {anova_table.loc[effect, 'Pr > F']}, Partial Eta Squared: {partial_eta_squared}")

# Running ANOVA for P200 amplitude
model_p200_amp = AnovaRM(dfAll.dropna(subset=['P200_Amplitude']), 'P200_Amplitude', 'Participant', within=['Ambiguity', 'Randomness'])
res_p200_amp = model_p200_amp.fit()
print("P200 Amplitude ANOVA")
print_anova_with_effect_sizes(res_p200_amp)

# Running ANOVA for P400 amplitude
model_p400_amp = AnovaRM(dfAll.dropna(subset=['P400_Amplitude']), 'P400_Amplitude', 'Participant', within=['Ambiguity', 'Randomness'])
res_p400_amp = model_p400_amp.fit()
print("P400 Amplitude ANOVA")
print_anova_with_effect_sizes(res_p400_amp)

# Running ANOVA for P200 latency
model_p200_lat = AnovaRM(dfAll.dropna(subset=['P200_Latency']), 'P200_Latency', 'Participant', within=['Ambiguity', 'Randomness'])
res_p200_lat = model_p200_lat.fit()
print("P200 Latency ANOVA")
print_anova_with_effect_sizes(res_p200_lat)

# Running ANOVA for P400 latency
model_p400_lat = AnovaRM(dfAll.dropna(subset=['P400_Latency']), 'P400_Latency', 'Participant', within=['Ambiguity', 'Randomness'])
res_p400_lat = model_p400_lat.fit()
print("P400 Latency ANOVA")
print_anova_with_effect_sizes(res_p400_lat)


P200 Amplitude ANOVA
Ambiguity - F Value: 1.3298226669454392, p-value: 0.2612124317259424, Partial Eta Squared: 0.057000976215287796
Randomness - F Value: 1.3793929049505596, p-value: 0.25276133531249273, Partial Eta Squared: 0.059000373130239606
Ambiguity:Randomness - F Value: 0.6273875695083952, p-value: 0.4367732227250336, Partial Eta Squared: 0.02772691136266359
P400 Amplitude ANOVA
Ambiguity - F Value: 40.201757695029166, p-value: 2.2179968774043187e-06, Partial Eta Squared: 0.6463122455821192
Randomness - F Value: 5.398963344931899, p-value: 0.02978383401305363, Partial Eta Squared: 0.19704991305558164
Ambiguity:Randomness - F Value: 0.7002700609131252, p-value: 0.41169111313140827, Partial Eta Squared: 0.03084853435813955
P200 Latency ANOVA
Ambiguity - F Value: 2.07813284987221, p-value: 0.16350704846934133, Partial Eta Squared: 0.08630789035135833
Randomness - F Value: 12.818597249989821, p-value: 0.0016687967614118081, Partial Eta Squared: 0.36815375294860764
Ambiguity:Randomn

C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\2222521068.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfAll = pd.concat([dfAll, entry], ignore_index=True)


In [20]:
"""Uncorrected Anova test with details for all participants"""

# Create an empty DataFrame
df2122 = pd.DataFrame(columns=['Participant', 'Condition', 'Ambiguity', 'Randomness', 'P200_Amplitude', 'P200_Latency', 'P400_Amplitude', 'P400_Latency'])

# Populate the DataFrame with the peak detection results
for participant, evokeds in all_parts_combined.items():
    for evoked in evokeds:
        peaks = detect_peaks(evoked)
        entry = pd.DataFrame({
            'Participant': [participant],
            'Condition': [evoked.comment],
            'Ambiguity': ['Ambiguous' if 'A' in evoked.comment else 'Unambiguous'],
            'Randomness': ['Random' if 'R' in evoked.comment else 'Steady State'],
            'P200_Amplitude': [peaks.get('P200_amplitude', np.nan)],  # use np.nan for missing data
            'P200_Latency': [peaks.get('P200_time', np.nan)],
            'P400_Amplitude': [peaks.get('P400_amplitude', np.nan)],
            'P400_Latency': [peaks.get('P400_time', np.nan)] 
        })
        
        df2122 = pd.concat([df2122, entry], ignore_index=True)

df2122['P200_Amplitude'] = df2122['P200_Amplitude'] * 1e6
df2122['P400_Amplitude'] = df2122['P400_Amplitude'] * 1e6

def calculate_partial_eta_squared(f_value, df_effect, df_error):
    return (f_value * df_effect) / (f_value * df_effect + df_error)

# Function to print ANOVA results with effect sizes
def print_anova_with_effect_sizes(res):
    anova_table = res.anova_table
    for effect in anova_table.index:
        f_value = anova_table.loc[effect, 'F Value']
        df_effect = anova_table.loc[effect, 'Num DF']
        df_error = anova_table.loc[effect, 'Den DF']
        partial_eta_squared = calculate_partial_eta_squared(f_value, df_effect, df_error)
        print(f"{effect} - F Value: {f_value}, p-value: {anova_table.loc[effect, 'Pr > F']}, Partial Eta Squared: {partial_eta_squared}")

# Running ANOVA for P200 amplitude
model_p200_amp = AnovaRM(df2122.dropna(subset=['P200_Amplitude']), 'P200_Amplitude', 'Participant', within=['Ambiguity', 'Randomness'])
res_p200_amp = model_p200_amp.fit()
print("P200 Amplitude ANOVA")
print_anova_with_effect_sizes(res_p200_amp)

# Running ANOVA for P400 amplitude
model_p400_amp = AnovaRM(df2122.dropna(subset=['P400_Amplitude']), 'P400_Amplitude', 'Participant', within=['Ambiguity', 'Randomness'])
res_p400_amp = model_p400_amp.fit()
print("P400 Amplitude ANOVA")
print_anova_with_effect_sizes(res_p400_amp)

# Running ANOVA for P200 latency
model_p200_lat = AnovaRM(df2122.dropna(subset=['P200_Latency']), 'P200_Latency', 'Participant', within=['Ambiguity', 'Randomness'])
res_p200_lat = model_p200_lat.fit()
print("P200 Latency ANOVA")
print_anova_with_effect_sizes(res_p200_lat)

# Running ANOVA for P400 latency
model_p400_lat = AnovaRM(df2122.dropna(subset=['P400_Latency']), 'P400_Latency', 'Participant', within=['Ambiguity', 'Randomness'])
res_p400_lat = model_p400_lat.fit()
print("P400 Latency ANOVA")
print_anova_with_effect_sizes(res_p400_lat)


P200 Amplitude ANOVA
Ambiguity - F Value: 1.3298226669454392, p-value: 0.2612124317259424, Partial Eta Squared: 0.057000976215287796
Randomness - F Value: 1.3793929049505596, p-value: 0.25276133531249273, Partial Eta Squared: 0.059000373130239606
Ambiguity:Randomness - F Value: 0.6273875695083952, p-value: 0.4367732227250336, Partial Eta Squared: 0.02772691136266359
P400 Amplitude ANOVA
Ambiguity - F Value: 40.201757695029166, p-value: 2.2179968774043187e-06, Partial Eta Squared: 0.6463122455821192
Randomness - F Value: 5.398963344931899, p-value: 0.02978383401305363, Partial Eta Squared: 0.19704991305558164
Ambiguity:Randomness - F Value: 0.7002700609131252, p-value: 0.41169111313140827, Partial Eta Squared: 0.03084853435813955
P200 Latency ANOVA
Ambiguity - F Value: 2.07813284987221, p-value: 0.16350704846934133, Partial Eta Squared: 0.08630789035135833
Randomness - F Value: 12.818597249989821, p-value: 0.0016687967614118081, Partial Eta Squared: 0.36815375294860764
Ambiguity:Randomn

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\2923938866.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2122 = pd.concat([df2122, entry], ignore_index=True)


In [19]:
"""Anova test with pingouin and group test only for dataset 2021-2022-2023"""

# Define participant groups
groups = {
    'Group1': ['KM', 'KN', 'KO', 'KP'],
    'Group2': ['BA', 'BC', 'BD'],
    'Group3': ['AA', 'AB', 'AC', 'AD', 'AF', 'AG', 'AH'],
    'Group4': ['AJ', 'AK', 'AL', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT']
}

# Create a reverse lookup dictionary for participant groups
participant_to_group = {}
for group_name, participants in groups.items():
    for participant in participants:
        participant_to_group[participant] = group_name

# Create an empty DataFrame
df2122 = pd.DataFrame(columns=['Participant', 'Group', 'Condition', 'Ambiguity', 'Randomness', 'P200_Amplitude', 'P200_Latency', 'P400_Amplitude', 'P400_Latency'])

# Populate the DataFrame with the peak detection results
for participant, evokeds in all_parts_combined.items():
    group = participant_to_group.get(participant, 'Unknown')  # Get group, default to 'Unknown'
    for evoked in evokeds:
        peaks = detect_peaks(evoked)
        entry = pd.DataFrame({
            'Participant': [participant],
            'Group': [group],
            'Condition': [evoked.comment],
            'Ambiguity': ['Ambiguous' if 'A' in evoked.comment else 'Unambiguous'],
            'Randomness': ['Random' if 'R' in evoked.comment else 'Steady State'],
            'P200_Amplitude': [peaks.get('P200_amplitude', np.nan)],
            'P200_Latency': [peaks.get('P200_time', np.nan)],
            'P400_Amplitude': [peaks.get('P400_amplitude', np.nan)],
            'P400_Latency': [peaks.get('P400_time', np.nan)]
        })
        df2122 = pd.concat([df2122, entry], ignore_index=True)

# print(dfAll.head())  # Print first few rows to check
df2122['P200_Amplitude'] = df2122['P200_Amplitude'] * 1e6
df2122['P400_Amplitude'] = df2122['P400_Amplitude'] * 1e6
# Run mixed ANOVA for P200 amplitude
aov_p200_amp = pg.mixed_anova(data=df2122.dropna(subset=['P200_Amplitude']), dv='P200_Amplitude', between='Group', within='Ambiguity', subject='Participant')
print("P200 Amplitude-Ambiguity ANOVA")
print(aov_p200_amp)

# Run mixed ANOVA for P400 amplitude
aov_p400_amp = pg.mixed_anova(data=df2122.dropna(subset=['P400_Amplitude']), dv='P400_Amplitude', between='Group', within='Ambiguity', subject='Participant')
print("P400 Amplitude-Ambiguity ANOVA")
print(aov_p400_amp)

# Run mixed ANOVA for P200 latency
aov_p200_lat = pg.mixed_anova(data=df2122.dropna(subset=['P200_Latency']), dv='P200_Latency', between='Group', within='Ambiguity', subject='Participant')
print("P200 Latency-Ambiguity ANOVA")
print(aov_p200_lat)

# Run mixed ANOVA for P400 latency
aov_p400_lat = pg.mixed_anova(data=df2122.dropna(subset=['P400_Latency']), dv='P400_Latency', between='Group', within='Ambiguity', subject='Participant')
print("P400 Latency-Ambiguity ANOVA")
print(aov_p400_lat)



# Run mixed ANOVA for P200 amplitude
aov_p200_amp = pg.mixed_anova(data=df2122.dropna(subset=['P200_Amplitude']), dv='P200_Amplitude', between='Group', within='Randomness', subject='Participant')
print("P200 Amplitude-Randomness ANOVA")
print(aov_p200_amp)

# Run mixed ANOVA for P400 amplitude
aov_p400_amp = pg.mixed_anova(data=df2122.dropna(subset=['P400_Amplitude']), dv='P400_Amplitude', between='Group', within='Randomness', subject='Participant')
print("P400 Amplitude-Randomness ANOVA")
print(aov_p400_amp)

# Run mixed ANOVA for P200 latency
aov_p200_lat = pg.mixed_anova(data=df2122.dropna(subset=['P200_Latency']), dv='P200_Latency', between='Group', within='Randomness', subject='Participant')
print("P200 Latency-Randomness ANOVA")
print(aov_p200_lat)

# Run mixed ANOVA for P400 latency
aov_p400_lat = pg.mixed_anova(data=df2122.dropna(subset=['P400_Latency']), dv='P400_Latency', between='Group', within='Randomness', subject='Participant')
print("P400 Latency-Randomness ANOVA")
print(aov_p400_lat)

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3574163733.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2122 = pd.concat([df2122, entry], ignore_index=True)


P200 Amplitude-Ambiguity ANOVA
        Source          SS  DF1  DF2         MS         F     p-unc       np2  \
0        Group  245.075556    3   19  81.691852  2.380436  0.101634  0.273181   
1    Ambiguity    4.743967    1   19   4.743967  1.460726  0.241642  0.071392   
2  Interaction   16.776232    3   19   5.592077  1.721870  0.196403  0.213759   

   eps  
0  NaN  
1  1.0  
2  NaN  
P400 Amplitude-Ambiguity ANOVA
        Source          SS  DF1  DF2          MS          F     p-unc  \
0        Group  287.722908    3   19   95.907636   3.231081  0.045459   
1    Ambiguity  165.977592    1   19  165.977592  36.856992  0.000008   
2  Interaction    5.267095    3   19    1.755698   0.389870  0.761625   

        np2  eps  
0  0.337823  NaN  
1  0.659846  1.0  
2  0.057989  NaN  
P200 Latency-Ambiguity ANOVA
        Source        SS  DF1  DF2        MS         F     p-unc       np2  \
0        Group  0.004698    3   19  0.001566  3.990485  0.023192  0.386532   
1    Ambiguity  0.00004

In [13]:
"""Anova test with pingouin and group test only for all participants"""

# Define participant groups
groups = {
    'Group1': ['KM', 'KN', 'KO', 'KP'],
    'Group2': ['BA', 'BC', 'BD'],
    'Group3': ['AA', 'AB', 'AC', 'AD', 'AF', 'AG', 'AH'],
    'Group4': ['AJ', 'AK', 'AL', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT']
}

# Create a reverse lookup dictionary for participant groups
participant_to_group = {}
for group_name, participants in groups.items():
    for participant in participants:
        participant_to_group[participant] = group_name

# Create an empty DataFrame
dfAll = pd.DataFrame(columns=['Participant', 'Group', 'Condition', 'Ambiguity', 'Randomness', 'P200_Amplitude', 'P200_Latency', 'P400_Amplitude', 'P400_Latency'])

# Populate the DataFrame with the peak detection results
for participant, evokeds in all_parts_combined.items():
    group = participant_to_group.get(participant, 'Unknown')  # Get group, default to 'Unknown'
    for evoked in evokeds:
        peaks = detect_peaks(evoked)
        entry = pd.DataFrame({
            'Participant': [participant],
            'Group': [group],
            'Condition': [evoked.comment],
            'Ambiguity': ['Ambiguous' if 'A' in evoked.comment else 'Unambiguous'],
            'Randomness': ['Random' if 'R' in evoked.comment else 'Steady State'],
            'P200_Amplitude': [peaks.get('P200_amplitude', np.nan)],
            'P200_Latency': [peaks.get('P200_time', np.nan)],
            'P400_Amplitude': [peaks.get('P400_amplitude', np.nan)],
            'P400_Latency': [peaks.get('P400_time', np.nan)]
        })
        dfAll = pd.concat([dfAll, entry], ignore_index=True)
        

# print(dfAll.head())  # Print first few rows to check
dfAll['P200_Amplitude'] = dfAll['P200_Amplitude'] * 1e6
dfAll['P400_Amplitude'] = dfAll['P400_Amplitude'] * 1e6
# Run mixed ANOVA for P200 amplitude
aov_p200_amp = pg.mixed_anova(data=dfAll.dropna(subset=['P200_Amplitude']), dv='P200_Amplitude', between='Group', within='Ambiguity', subject='Participant')
print("P200 Amplitude-Ambiguity ANOVA")
print(aov_p200_amp)

# Run mixed ANOVA for P400 amplitude
aov_p400_amp = pg.mixed_anova(data=dfAll.dropna(subset=['P400_Amplitude']), dv='P400_Amplitude', between='Group', within='Ambiguity', subject='Participant')
print("P400 Amplitude-Ambiguity ANOVA")
print(aov_p400_amp)

# Run mixed ANOVA for P200 latency
aov_p200_lat = pg.mixed_anova(data=dfAll.dropna(subset=['P200_Latency']), dv='P200_Latency', between='Group', within='Ambiguity', subject='Participant')
print("P200 Latency-Ambiguity ANOVA")
print(aov_p200_lat)

# Run mixed ANOVA for P400 latency
aov_p400_lat = pg.mixed_anova(data=dfAll.dropna(subset=['P400_Latency']), dv='P400_Latency', between='Group', within='Ambiguity', subject='Participant')
print("P400 Latency-Ambiguity ANOVA")
print(aov_p400_lat)



# Run mixed ANOVA for P200 amplitude
aov_p200_amp = pg.mixed_anova(data=dfAll.dropna(subset=['P200_Amplitude']), dv='P200_Amplitude', between='Group', within='Randomness', subject='Participant')
print("P200 Amplitude-Randomness ANOVA")
print(aov_p200_amp)

# Run mixed ANOVA for P400 amplitude
aov_p400_amp = pg.mixed_anova(data=dfAll.dropna(subset=['P400_Amplitude']), dv='P400_Amplitude', between='Group', within='Randomness', subject='Participant')
print("P400 Amplitude-Randomness ANOVA")
print(aov_p400_amp)

# Run mixed ANOVA for P200 latency
aov_p200_lat = pg.mixed_anova(data=dfAll.dropna(subset=['P200_Latency']), dv='P200_Latency', between='Group', within='Randomness', subject='Participant')
print("P200 Latency-Randomness ANOVA")
print(aov_p200_lat)

# Run mixed ANOVA for P400 latency
aov_p400_lat = pg.mixed_anova(data=dfAll.dropna(subset=['P400_Latency']), dv='P400_Latency', between='Group', within='Randomness', subject='Participant')
print("P400 Latency-Randomness ANOVA")
print(aov_p400_lat)

C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\2260176270.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfAll = pd.concat([dfAll, entry], ignore_index=True)


P200 Amplitude-Ambiguity ANOVA
        Source          SS  DF1  DF2         MS         F     p-unc       np2  \
0        Group  245.075556    3   19  81.691852  2.380436  0.101634  0.273181   
1    Ambiguity    4.743967    1   19   4.743967  1.460726  0.241642  0.071392   
2  Interaction   16.776232    3   19   5.592077  1.721870  0.196403  0.213759   

   eps  
0  NaN  
1  1.0  
2  NaN  
P400 Amplitude-Ambiguity ANOVA
        Source          SS  DF1  DF2          MS          F     p-unc  \
0        Group  287.722908    3   19   95.907636   3.231081  0.045459   
1    Ambiguity  165.977592    1   19  165.977592  36.856992  0.000008   
2  Interaction    5.267095    3   19    1.755698   0.389870  0.761625   

        np2  eps  
0  0.337823  NaN  
1  0.659846  1.0  
2  0.057989  NaN  
P200 Latency-Ambiguity ANOVA
        Source        SS  DF1  DF2        MS         F     p-unc       np2  \
0        Group  0.004698    3   19  0.001566  3.990485  0.023192  0.386532   
1    Ambiguity  0.00004

In [50]:
"""Create boxplot for amplitude and latency values"""

# Create boxplots for P200 Amplitude
plt.figure(figsize=(12, 6))
sns.boxplot(x='Condition', y='P200_Amplitude', data=dfAll, palette={'US': 'darkred', 'AS': 'darkblue', 'UR': 'red', 'AR': 'blue'}, showfliers=False)
sns.stripplot(x='Condition', y='P200_Amplitude', data=dfAll, color='green', jitter=0.2, size=5)  # Increased size
plt.title('P200 Amplitude by Condition', fontsize=16)
plt.xlabel('Condition', fontsize=14)
plt.ylabel('P200 Amplitude (µV)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# Create boxplots for P400 Amplitude
plt.figure(figsize=(12, 6))
sns.boxplot(x='Condition', y='P400_Amplitude', data=dfAll, palette={'US': 'darkred', 'AS': 'darkblue', 'UR': 'red', 'AR': 'blue'}, showfliers=False)
sns.stripplot(x='Condition', y='P400_Amplitude', data=dfAll, color='green', jitter=0.2, size=5)  # Increased size
plt.title('P400 Amplitude by Condition', fontsize=16)
plt.xlabel('Condition', fontsize=14)
plt.ylabel('P400 Amplitude (µV)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# Create boxplots for P200 Latency
plt.figure(figsize=(12, 6))
sns.boxplot(x='Condition', y='P200_Latency', data=dfAll, palette={'US': 'darkred', 'AS': 'darkblue', 'UR': 'red', 'AR': 'blue'}, showfliers=False)
sns.stripplot(x='Condition', y='P200_Latency', data=dfAll, color='green', jitter=0.2, size=5)  # Increased size
plt.title('P200 Latency by Condition', fontsize=16)
plt.xlabel('Condition', fontsize=14)
plt.ylabel('P200 Latency (ms)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# Create boxplots for P400 Latency
plt.figure(figsize=(12, 6))
sns.boxplot(x='Condition', y='P400_Latency', data=dfAll, palette={'US': 'darkred', 'AS': 'darkblue', 'UR': 'red', 'AR': 'blue'}, showfliers=False)
sns.stripplot(x='Condition', y='P400_Latency', data=dfAll, color='green', jitter=0.2, size=5)  # Increased size
plt.title('P400 Latency by Condition', fontsize=16)
plt.xlabel('Condition', fontsize=14)
plt.ylabel('P400 Latency (ms)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\3514180731.py:7: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Condition', y='P200_Amplitude', data=dfAll, palette={'US': 'darkred', 'AS': 'darkblue', 'UR': 'red', 'AR': 'blue'}, showfliers=False)
c:\Python311\Lib\site-packages\seaborn\_base.py:948: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
c:\Python311\Lib\site-packages\seaborn\categorical.py:632: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)
c:\Python311\Lib\site-packages\seaborn\_base.py:948: FutureWarning: When grou

In [51]:
"""Boxplots with p-values"""

# Define a color palette
color_palette_ambiguity = {'Ambiguous': 'blue', 'Unambiguous': 'red'}
color_palette_randomness = {'Random': 'green', 'Steady State': 'orange'}

# Function to add p-values in the middle of the plot
def add_p_value(text, ax):
    # Get the y-limit of the current plot
    y_min, y_max = ax.get_ylim()
    # Calculate the vertical position (middle of the plot)
    y_pos = (y_max + y_min) / 2
    # Add text annotation
    ax.text(0.5, y_pos, text, ha='center', va='center', fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.6))

# Create boxplots for P200 Amplitude grouped by Ambiguity
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Ambiguity', y='P200_Amplitude', data=dfAll, palette=color_palette_ambiguity, showfliers=False)
sns.stripplot(x='Ambiguity', y='P200_Amplitude', data=dfAll, color='black', jitter=0.2, size=5)
plt.title('P200 Amplitude by Ambiguity', fontsize=16)
plt.xlabel('Ambiguity', fontsize=14)
plt.ylabel('P200 Amplitude (µV)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
add_p_value('p-value: 0.26', ax)
plt.show()

# Create boxplots for P400 Amplitude grouped by Ambiguity
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Ambiguity', y='P400_Amplitude', data=dfAll, palette=color_palette_ambiguity, showfliers=False)
sns.stripplot(x='Ambiguity', y='P400_Amplitude', data=dfAll, color='black', jitter=0.2, size=5)
plt.title('P400 Amplitude by Ambiguity', fontsize=16)
plt.xlabel('Ambiguity', fontsize=14)
plt.ylabel('P400 Amplitude (µV)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
add_p_value('p-value: 0.000008, corrected p-value: 0.000009', ax)
plt.show()

# Create boxplots for P200 Amplitude grouped by Randomness
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Randomness', y='P200_Amplitude', data=dfAll, palette=color_palette_randomness, showfliers=False)
sns.stripplot(x='Randomness', y='P200_Amplitude', data=dfAll, color='black', jitter=0.2, size=5)
plt.title('P200 Amplitude by Randomness', fontsize=16)
plt.xlabel('Randomness', fontsize=14)
plt.ylabel('P200 Amplitude (µV)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
add_p_value('p-value: 0.25', ax)
plt.show()

# Create boxplots for P400 Amplitude grouped by Randomness
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Randomness', y='P400_Amplitude', data=dfAll, palette=color_palette_randomness, showfliers=False)
sns.stripplot(x='Randomness', y='P400_Amplitude', data=dfAll, color='black', jitter=0.2, size=5)
plt.title('P400 Amplitude by Randomness', fontsize=16)
plt.xlabel('Randomness', fontsize=14)
plt.ylabel('P400 Amplitude (µV)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
add_p_value('p-value: 0.02, corrected p-value: 0.1', ax)
plt.show()

# Create boxplots for P200 Latency grouped by Ambiguity
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Ambiguity', y='P200_Latency', data=dfAll, palette=color_palette_ambiguity, showfliers=False)
sns.stripplot(x='Ambiguity', y='P200_Latency', data=dfAll, color='black', jitter=0.2, size=5)
plt.title('P200 Latency by Ambiguity', fontsize=16)
plt.xlabel('Ambiguity', fontsize=14)
plt.ylabel('P200 Latency (ms)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
add_p_value('p-value: 0.16', ax)
plt.show()

# Create boxplots for P400 Latency grouped by Ambiguity
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Ambiguity', y='P400_Latency', data=dfAll, palette=color_palette_ambiguity, showfliers=False)
sns.stripplot(x='Ambiguity', y='P400_Latency', data=dfAll, color='black', jitter=0.2, size=5)
plt.title('P400 Latency by Ambiguity', fontsize=16)
plt.xlabel('Ambiguity', fontsize=14)
plt.ylabel('P400 Latency (ms)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
add_p_value('p-value: 0.004, corrected p-value: 0.2', ax)
plt.show()

# Create boxplots for P200 Latency grouped by Randomness
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Randomness', y='P200_Latency', data=dfAll, palette=color_palette_randomness, showfliers=False)
sns.stripplot(x='Randomness', y='P200_Latency', data=dfAll, color='black', jitter=0.2, size=5)
plt.title('P200 Latency by Randomness', fontsize=16)
plt.xlabel('Randomness', fontsize=14)
plt.ylabel('P200 Latency (ms)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
add_p_value('p-value: 0.002, corrected p-value: 0.006', ax)
plt.show()

# Create boxplots for P400 Latency grouped by Randomness
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Randomness', y='P400_Latency', data=dfAll, palette=color_palette_randomness, showfliers=False)
sns.stripplot(x='Randomness', y='P400_Latency', data=dfAll, color='black', jitter=0.2, size=5)
plt.title('P400 Latency by Randomness', fontsize=16)
plt.xlabel('Randomness', fontsize=14)
plt.ylabel('P400 Latency (ms)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
add_p_value('p-value: 0.35', ax)
plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\3590523954.py:16: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.boxplot(x='Ambiguity', y='P200_Amplitude', data=dfAll, palette=color_palette_ambiguity, showfliers=False)
c:\Python311\Lib\site-packages\seaborn\_base.py:948: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
c:\Python311\Lib\site-packages\seaborn\categorical.py:632: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)
c:\Python311\Lib\site-packages\seaborn\_base.py:948: FutureWarning: When grouping with a length-1 list-like, y

In [23]:
dfAll

,Participant,Group,Condition,Ambiguity,Randomness,P200_Amplitude,P200_Latency,P400_Amplitude,P400_Latency
0,BA,Group2,US,Unambiguous,Study State,3.369963,0.241,4.178744,0.322
1,BA,Group2,AS,Ambiguous,Study State,2.213954,0.231,1.350312,0.306
2,BA,Group2,UR,Unambiguous,Random,4.139553,0.232,5.437677,0.420
3,BA,Group2,AR,Ambiguous,Random,3.356740,0.236,3.017965,0.326
4,BC,Group2,US,Unambiguous,Study State,12.390849,0.222,3.106674,0.418
...,...,...,...,...,...,...,...,...,...
87,AS,Group4,AR,Ambiguous,Random,12.565052,0.229,5.598564,0.333
88,AT,Group4,US,Unambiguous,Study State,13.063968,0.246,8.247790,0.423
89,AT,Group4,AS,Ambiguous,Study State,9.903590,0.240,7.837134,0.447
90,AT,Group4,UR,Unambiguous,Random,8.314536,0.247,5.482178,0.448


In [21]:
print(df2122)

   Participant Condition    Ambiguity    Randomness  P200_Amplitude  \
0           BA        US  Unambiguous  Steady State        3.369963   
1           BA        AS    Ambiguous  Steady State        2.213954   
2           BA        UR  Unambiguous        Random        4.139553   
3           BA        AR    Ambiguous        Random        3.356740   
4           BC        US  Unambiguous  Steady State       12.390849   
..         ...       ...          ...           ...             ...   
87          AS        AR    Ambiguous        Random       12.565052   
88          AT        US  Unambiguous  Steady State       13.063968   
89          AT        AS    Ambiguous  Steady State        9.903590   
90          AT        UR  Unambiguous        Random        8.314536   
91          AT        AR    Ambiguous        Random        6.197472   

    P200_Latency  P400_Amplitude  P400_Latency  
0          0.241        4.178744         0.322  
1          0.231        1.350312         0.306  


In [29]:
"""Scatter plot for P200 amplitude"""
# Create a color map for participants
participants = df2122['Participant'].unique()  # Extract participants
color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed

# Create a dictionary to map each participant to a color
participant_colors = {participant: color_map(i) for i, participant in enumerate(participants)}

# Filter data for plotting
unambiguous_study_state_data = df2122[(df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Steady State')]
ambiguous_study_state_data = df2122[(df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Steady State')]
unambiguous_random_data = df2122[(df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Random')]
ambiguous_random_data = df2122[(df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Random')]

# Plotting
plt.figure(figsize=(5, 5))
marker_size = 100  # Increased marker size
for participant in participants:
    # Filter by participant
    part_unamb_study = unambiguous_study_state_data[unambiguous_study_state_data['Participant'] == participant]['P200_Amplitude']
    part_amb_study = ambiguous_study_state_data[ambiguous_study_state_data['Participant'] == participant]['P200_Amplitude']
    part_unamb_random = unambiguous_random_data[unambiguous_random_data['Participant'] == participant]['P200_Amplitude']
    part_amb_random = ambiguous_random_data[ambiguous_random_data['Participant'] == participant]['P200_Amplitude']
    
    # Ensure there are data points to plot
    min_study = min(len(part_unamb_study), len(part_amb_study))
    min_random = min(len(part_unamb_random), len(part_amb_random))
    
    # Plotting each participant's data with specific color
    plt.scatter(part_amb_study[:min_study], part_unamb_study[:min_study], color=participant_colors[participant], marker='o', s=marker_size)
    plt.scatter(part_amb_random[:min_random], part_unamb_random[:min_random], color=participant_colors[participant], marker='x', s=marker_size)

# Calculate limits based on data
buffer = 0.25
all_ambiguity = df2122['P200_Amplitude'].dropna()
all_unambiguity = df2122['P200_Amplitude'].dropna()
xlim_min = min(all_ambiguity.min(), all_unambiguity.min()) - buffer
xlim_max = max(all_ambiguity.max(), all_unambiguity.max()) + buffer
ylim_min = xlim_min  # Assuming symmetry for simplicity
ylim_max = xlim_max 
# Plot diagonal line
plt.plot([xlim_min, xlim_max], [ylim_min, ylim_max], color='brown', linestyle='--')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)

# Enhance the legend to show unique entries only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels

# Custom legend entries for 'o' and 'x' markers
custom_legend_entries = [plt.Line2D([0], [0], marker='o', color='r', markerfacecolor='k', markersize=10, label='Steady State'),
                         plt.Line2D([0], [0], marker='x', color='b', markerfacecolor='k', markersize=10, label='Random')]

# Combine custom legend entries with unique participant entries
combined_legend = custom_legend_entries + list(by_label.values())

# Create combined legend
legend_labels = [entry.get_label() for entry in custom_legend_entries] + list(by_label.keys())
legend = plt.legend(combined_legend, legend_labels, loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0., markerscale=0.7, title_fontsize='small')
plt.setp(legend.get_texts(), fontsize='x-small')

plt.xlabel('Ambiguous Amplitude (µV)', fontsize=16)
plt.ylabel('Unambiguous Amplitude (µV)', fontsize=16)
plt.title('P200 Peak Amplitudes', fontsize=16)
plt.grid(True)
plt.tight_layout()  # Adjust the layout to make room for the legend
# Add Figure name under the X axis label
plt.text((xlim_min + xlim_max) / 2, ylim_min - (ylim_max - ylim_min) * 0.15, 'Plot D', ha='center', fontsize=12)
plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\740432192.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed


In [28]:
"""Scatter plot for P400 amplitude, Ambiguous vs Unambiguous"""
# Create a color map for participants
participants = df2122['Participant'].unique()  # Extract participants
color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed

# Create a dictionary to map each participant to a color
participant_colors = {participant: color_map(i) for i, participant in enumerate(participants)}

# Filter data for plotting
unambiguous_study_state_data = df2122[(df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Steady State')]
ambiguous_study_state_data = df2122[(df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Steady State')]
unambiguous_random_data = df2122[(df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Random')]
ambiguous_random_data = df2122[(df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Random')]

# Plotting
plt.figure(figsize=(5, 5))
marker_size = 75  # Increased marker size
for participant in participants:
    # Filter by participant
    part_unamb_study = unambiguous_study_state_data[unambiguous_study_state_data['Participant'] == participant]['P400_Amplitude']
    part_amb_study = ambiguous_study_state_data[ambiguous_study_state_data['Participant'] == participant]['P400_Amplitude']
    part_unamb_random = unambiguous_random_data[unambiguous_random_data['Participant'] == participant]['P400_Amplitude']
    part_amb_random = ambiguous_random_data[ambiguous_random_data['Participant'] == participant]['P400_Amplitude']
    
    # Ensure there are data points to plot
    min_study = min(len(part_unamb_study), len(part_amb_study))
    min_random = min(len(part_unamb_random), len(part_amb_random))
    
    # Plotting each participant's data with specific color
    plt.scatter(part_amb_study[:min_study], part_unamb_study[:min_study], color=participant_colors[participant], marker='o', s=marker_size)
    plt.scatter(part_amb_random[:min_random], part_unamb_random[:min_random], color=participant_colors[participant], marker='x', s=marker_size)

# Calculate limits based on data
buffer = 0.5
all_ambiguity = df2122['P400_Amplitude'].dropna()
all_unambiguity = df2122['P400_Amplitude'].dropna()
xlim_min = min(all_ambiguity.min(), all_unambiguity.min()) - buffer
xlim_max = max(all_ambiguity.max(), all_unambiguity.max()) + buffer
ylim_min = xlim_min  # Assuming symmetry for simplicity
ylim_max = xlim_max 
# Plot diagonal line
plt.plot([xlim_min, xlim_max], [ylim_min, ylim_max], color='brown', linestyle='--')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)

# Enhance the legend to show unique entries only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels

# Custom legend entries for 'o' and 'x' markers
custom_legend_entries = [plt.Line2D([0], [0], marker='o', color='r', markerfacecolor='k', markersize=10, label='Steady State'),
                         plt.Line2D([0], [0], marker='x', color='b', markerfacecolor='k', markersize=10, label='Random')]

# Combine custom legend entries with unique participant entries
combined_legend = custom_legend_entries + list(by_label.values())

# Create combined legend
legend_labels = [entry.get_label() for entry in custom_legend_entries] + list(by_label.keys())

legend = plt.legend(combined_legend, legend_labels, loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0., markerscale=0.7, title_fontsize='small')
plt.setp(legend.get_texts(), fontsize='x-small')

plt.xlabel('Ambiguous Amplitude (µV)', fontsize=16)
plt.ylabel('Unambiguous Amplitude (µV)', fontsize=16)
plt.title('P400 Peak Amplitudes', fontsize=16)
plt.grid(True)
plt.tight_layout()  # Adjust the layout to make room for the legend
# Add Figure name under the X axis label
plt.text((xlim_min + xlim_max) / 2, ylim_min - (ylim_max - ylim_min) * 0.15, 'Plot E', ha='center', fontsize=12)
plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1835408799.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed


In [36]:
"""Scatter plot for P400 amplitude, Steady state vs Random"""

# Create a color map for participants
participants = df2122['Participant'].unique()  # Extract participants
color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed

# Create a dictionary to map each participant to a color
participant_colors = {participant: color_map(i) for i, participant in enumerate(participants)}

# Filter data for plotting
steady_state_data = df2122[df2122['Randomness'] == 'Steady State']
random_data = df2122[df2122['Randomness'] == 'Random']

# Plotting
plt.figure(figsize=(5, 5))
marker_size = 100  # Increased marker size
for participant in participants:
    # Filter by participant and ambiguity
    part_steady_unamb = steady_state_data[(steady_state_data['Participant'] == participant) & (steady_state_data['Ambiguity'] == 'Unambiguous')]['P400_Amplitude']
    part_steady_amb = steady_state_data[(steady_state_data['Participant'] == participant) & (steady_state_data['Ambiguity'] == 'Ambiguous')]['P400_Amplitude']
    part_random_unamb = random_data[(random_data['Participant'] == participant) & (random_data['Ambiguity'] == 'Unambiguous')]['P400_Amplitude']
    part_random_amb = random_data[(random_data['Participant'] == participant) & (random_data['Ambiguity'] == 'Ambiguous')]['P400_Amplitude']
    
    # Ensure there are data points to plot
    min_steady_unamb = min(len(part_steady_unamb), len(part_random_unamb))
    min_steady_amb = min(len(part_steady_amb), len(part_random_amb))
    
    # Plotting each participant's data with specific color and marker
    plt.scatter(part_steady_unamb[:min_steady_unamb], part_random_unamb[:min_steady_unamb], color=participant_colors[participant], marker='+', s=marker_size, label=participant if 'Unambiguous' in participant else "")
    plt.scatter(part_steady_amb[:min_steady_amb], part_random_amb[:min_steady_amb], color=participant_colors[participant], marker='^', s=marker_size, label=participant if 'Ambiguous' in participant else "")

# Calculate limits based on data
buffer = 0.5
all_steady = steady_state_data['P400_Amplitude'].dropna()
all_random = random_data['P400_Amplitude'].dropna()
xlim_min = min(all_steady.min(), all_random.min()) - buffer
xlim_max = max(all_steady.max(), all_random.max()) + buffer
ylim_min = xlim_min  # Assuming symmetry for simplicity
ylim_max = xlim_max 

# Plot diagonal line
plt.plot([xlim_min, xlim_max], [ylim_min, ylim_max], color='brown', linestyle='--')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)

# Enhance the legend to show unique entries only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels




# Custom legend entries for 'o' and 'x' markers
custom_legend_entries = [plt.Line2D([0], [0], marker='+', linestyle='None', color='g', markerfacecolor='k', markersize=10, label='Unambiguous'),
                         plt.Line2D([0], [0], marker='^', linestyle='None', color='m', markerfacecolor='k', markersize=10, label='Ambiguous')]

# Combine custom legend entries with unique participant entries
combined_legend = custom_legend_entries + list(by_label.values())

# Create combined legend
legend_labels = [entry.get_label() for entry in custom_legend_entries] + list(by_label.keys())

legend = plt.legend(combined_legend, legend_labels, loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0., markerscale=0.7, title_fontsize='small')
plt.setp(legend.get_texts(), fontsize='x-small')

plt.xlabel('Steady State Amplitude (µV)', fontsize=16)
plt.ylabel('Random Amplitude (µV)', fontsize=16)
plt.title('P400 Peak Amplitudes', fontsize=16)
plt.grid(True)
plt.tight_layout()  # Adjust the layout to make room for the legend

# Add Figure name under the X axis label
plt.text((xlim_min + xlim_max) / 2, ylim_min - (ylim_max - ylim_min) * 0.15, 'Plot F', ha='center', fontsize=12)

plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\1238685167.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed


In [38]:
"""Scatter plot for P400 latency, Steady state vs Random"""
# Create a color map for participants
participants = df2122['Participant'].unique()  # Extract participants
color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed

# Create a dictionary to map each participant to a color
participant_colors = {participant: color_map(i) for i, participant in enumerate(participants)}

# Filter data for plotting
steady_state_data = df2122[df2122['Randomness'] == 'Steady State']
random_data = df2122[df2122['Randomness'] == 'Random']

# Plotting
plt.figure(figsize=(5, 5))
marker_size = 100  # Increased marker size
for participant in participants:
    # Filter by participant and ambiguity
    part_steady_unamb = steady_state_data[(steady_state_data['Participant'] == participant) & (steady_state_data['Ambiguity'] == 'Unambiguous')]['P400_Latency']
    part_steady_amb = steady_state_data[(steady_state_data['Participant'] == participant) & (steady_state_data['Ambiguity'] == 'Ambiguous')]['P400_Latency']
    part_random_unamb = random_data[(random_data['Participant'] == participant) & (random_data['Ambiguity'] == 'Unambiguous')]['P400_Latency']
    part_random_amb = random_data[(random_data['Participant'] == participant) & (random_data['Ambiguity'] == 'Ambiguous')]['P400_Latency']
    
    # Ensure there are data points to plot
    min_steady_unamb = min(len(part_steady_unamb), len(part_random_unamb))
    min_steady_amb = min(len(part_steady_amb), len(part_random_amb))
    
    # Plotting each participant's data with specific color and marker
    plt.scatter(part_steady_unamb[:min_steady_unamb], part_random_unamb[:min_steady_unamb], color=participant_colors[participant], marker='+', s=marker_size, label=participant if 'Unambiguous' in participant else "")
    plt.scatter(part_steady_amb[:min_steady_amb], part_random_amb[:min_steady_amb], color=participant_colors[participant], marker='^', s=marker_size, label=participant if 'Ambiguous' in participant else "")

# Calculate limits based on data
buffer = 0.01
all_steady = steady_state_data['P400_Latency'].dropna()
all_random = random_data['P400_Latency'].dropna()
xlim_min = min(all_steady.min(), all_random.min()) - buffer
xlim_max = max(all_steady.max(), all_random.max()) + buffer
ylim_min = xlim_min  # Assuming symmetry for simplicity
ylim_max = xlim_max 

# Plot diagonal line
plt.plot([xlim_min, xlim_max], [ylim_min, ylim_max], color='brown', linestyle='--')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)

# Enhance the legend to show unique entries only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels



# Custom legend entries for 'o' and 'x' markers
custom_legend_entries = [plt.Line2D([0], [0], marker='+', linestyle='None', color='g', markerfacecolor='k', markersize=10, label='Unambiguous'),
                         plt.Line2D([0], [0], marker='^', linestyle='None', color='m', markerfacecolor='k', markersize=10, label='Ambiguous')]

# Combine custom legend entries with unique participant entries
combined_legend = custom_legend_entries + list(by_label.values())

# Create combined legend
legend_labels = [entry.get_label() for entry in custom_legend_entries] + list(by_label.keys())

legend = plt.legend(combined_legend, legend_labels, loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0., markerscale=0.7, title_fontsize='small')
plt.setp(legend.get_texts(), fontsize='x-small')

plt.xlabel('Steady State Latencies (ms)', fontsize=16)
plt.ylabel('Random Latencies (ms)', fontsize=16)
plt.title('P400 Peak Latencies', fontsize=16)
plt.grid(True)
plt.tight_layout()  # Adjust the layout to make room for the legend

# Add Figure name under the X axis label
plt.text((xlim_min + xlim_max) / 2, ylim_min - (ylim_max - ylim_min) * 0.15, 'Plot D', ha='center', fontsize=12)

plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\2665826161.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed


In [39]:
"""Scatter plot for P200 latency, Steady state vs Random"""
# Create a color map for participants
participants = df2122['Participant'].unique()  # Extract participants
color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed

# Create a dictionary to map each participant to a color
participant_colors = {participant: color_map(i) for i, participant in enumerate(participants)}

# Filter data for plotting
steady_state_data = df2122[df2122['Randomness'] == 'Steady State']
random_data = df2122[df2122['Randomness'] == 'Random']

# Plotting
plt.figure(figsize=(5, 5))
marker_size = 100  # Increased marker size
for participant in participants:
    # Filter by participant and ambiguity
    part_steady_unamb = steady_state_data[(steady_state_data['Participant'] == participant) & (steady_state_data['Ambiguity'] == 'Unambiguous')]['P200_Latency']
    part_steady_amb = steady_state_data[(steady_state_data['Participant'] == participant) & (steady_state_data['Ambiguity'] == 'Ambiguous')]['P200_Latency']
    part_random_unamb = random_data[(random_data['Participant'] == participant) & (random_data['Ambiguity'] == 'Unambiguous')]['P200_Latency']
    part_random_amb = random_data[(random_data['Participant'] == participant) & (random_data['Ambiguity'] == 'Ambiguous')]['P200_Latency']
    
    # Ensure there are data points to plot
    min_steady_unamb = min(len(part_steady_unamb), len(part_random_unamb))
    min_steady_amb = min(len(part_steady_amb), len(part_random_amb))
    
    # Plotting each participant's data with specific color and marker
    plt.scatter(part_steady_unamb[:min_steady_unamb], part_random_unamb[:min_steady_unamb], color=participant_colors[participant], marker='+', s=marker_size, label=participant if 'Unambiguous' in participant else "")
    plt.scatter(part_steady_amb[:min_steady_amb], part_random_amb[:min_steady_amb], color=participant_colors[participant], marker='^', s=marker_size, label=participant if 'Ambiguous' in participant else "")

# Calculate limits based on data
buffer = 0.001
all_steady = steady_state_data['P200_Latency'].dropna()
all_random = random_data['P200_Latency'].dropna()
xlim_min = min(all_steady.min(), all_random.min()) - buffer
xlim_max = max(all_steady.max(), all_random.max()) + buffer
ylim_min = xlim_min  # Assuming symmetry for simplicity
ylim_max = xlim_max 

# Plot diagonal line
plt.plot([xlim_min, xlim_max], [ylim_min, ylim_max], color='brown', linestyle='--')
plt.xlim(xlim_min, xlim_max)
plt.ylim(ylim_min, ylim_max)

# Enhance the legend to show unique entries only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels


# Custom legend entries for 'o' and 'x' markers
custom_legend_entries = [plt.Line2D([0], [0], marker='+', linestyle='None', color='g', markerfacecolor='k', markersize=10, label='Unambiguous'),
                         plt.Line2D([0], [0], marker='^', linestyle='None', color='m', markerfacecolor='k', markersize=10, label='Ambiguous')]

# Combine custom legend entries with unique participant entries
combined_legend = custom_legend_entries + list(by_label.values())

# Create combined legend
legend_labels = [entry.get_label() for entry in custom_legend_entries] + list(by_label.keys())

legend = plt.legend(combined_legend, legend_labels, loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0., markerscale=0.7, title_fontsize='small')
plt.setp(legend.get_texts(), fontsize='x-small')

plt.xlabel('Steady State Latencies (ms)', fontsize=16)
plt.ylabel('Random Latencies (ms)', fontsize=16)
plt.title('P200 Peak Latencies', fontsize=16)
plt.grid(True)
plt.tight_layout()  # Adjust the layout to make room for the legend

# Add Figure name under the X axis label
plt.text((xlim_min + xlim_max) / 2, ylim_min - (ylim_max - ylim_min) * 0.15, 'Plot C', ha='center', fontsize=12)

plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_25248\3786465566.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed


In [69]:
"""Value differences"""

# Create a color map for participants
participants = df2122['Participant'].unique()  # Extract unique participants
color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed

# Create a dictionary to map each participant to a color
participant_colors = {participant: color_map(i) for i, participant in enumerate(participants)}

# Initializing lists to store amplitude differences
study_state_latency_diff = []
random_latency_diff = []

# Calculating differences
for participant in participants:
    # Filter by participant
    unamb_study_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Steady State')]['P200_Latency'].mean()
    amb_study_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Steady State')]['P200_Latency'].mean()
    unamb_random_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Random')]['P200_Latency'].mean()
    amb_random_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Random')]['P200_Latency'].mean()
    
    # Calculate differences and append to lists
    study_state_latency_diff.append(unamb_study_amplitude - amb_study_amplitude)
    random_latency_diff.append(unamb_random_amplitude - amb_random_amplitude)

# Plotting
plt.figure(figsize=(10, 10))
for i, participant in enumerate(participants):
    plt.scatter(study_state_latency_diff[i], random_latency_diff[i], color=participant_colors[participant], label=f' {participant}')
# Calculate limits based on data
buffer = 0.001
xlim_min = min(study_state_latency_diff) - buffer
xlim_max = max(study_state_latency_diff) + buffer
ylim_min = min(random_latency_diff) - buffer
ylim_max = max(random_latency_diff) + buffer

# Ensure the plot is square by setting the same limits for both axes
combined_min = min(xlim_min, ylim_min)
combined_max = max(xlim_max, ylim_max)
plt.plot([combined_min, combined_max], [combined_min, combined_max], color='brown', linestyle='--')
plt.xlim(combined_min, combined_max)
plt.ylim(combined_min, combined_max)
# Enhance the legend to show unique entries only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0.)

plt.xlabel('Steady State Latency Difference (Unambiguous - Ambiguous)')
plt.ylabel('Random State Latency Difference (Unambiguous - Ambiguous)')
plt.title('Scatter Plot of Latency Differences at P200 by Participant')
plt.grid(True)
plt.tight_layout()  # Adjust the layout to make room for the legend
plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\2084213329.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed


In [70]:

"""Value differences"""
# Create a color map for participants
participants = df2122['Participant'].unique()  # Extract unique participants
color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed

# Create a dictionary to map each participant to a color
participant_colors = {participant: color_map(i) for i, participant in enumerate(participants)}

# Initializing lists to store amplitude differences
study_state_amplitude_diff = []
random_amplitude_diff = []

# Calculating differences
for participant in participants:
    # Filter by participant
    unamb_study_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Steady State')]['P400_Amplitude'].mean()
    amb_study_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Steady State')]['P400_Amplitude'].mean()
    unamb_random_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Random')]['P400_Amplitude'].mean()
    amb_random_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Random')]['P400_Amplitude'].mean()
    
    # Calculate differences and append to lists
    study_state_amplitude_diff.append(unamb_study_amplitude - amb_study_amplitude)
    random_amplitude_diff.append(unamb_random_amplitude - amb_random_amplitude)

# Plotting
plt.figure(figsize=(10, 10))
for i, participant in enumerate(participants):
    plt.scatter(study_state_amplitude_diff[i], random_amplitude_diff[i], color=participant_colors[participant], label=f' {participant}')
# Calculate limits based on data
buffer = 0.1
xlim_min = min(study_state_amplitude_diff)
xlim_max = max(study_state_amplitude_diff)
ylim_min = min(random_amplitude_diff)
ylim_max = max(random_amplitude_diff) + buffer

# Ensure the plot is square by setting the same limits for both axes
combined_min = min(xlim_min, ylim_min)
combined_max = max(xlim_max, ylim_max)
plt.plot([combined_min, combined_max], [combined_min, combined_max], color='brown', linestyle='--')
plt.xlim(combined_min, combined_max)
plt.ylim(combined_min, combined_max)
# Enhance the legend to show unique entries only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0.)

plt.xlabel('Steady State Amplitude Difference (Unambiguous - Ambiguous)')
plt.ylabel('Random State Amplitude Difference (Unambiguous - Ambiguous)')
plt.title('Scatter Plot of Amplitude Differences at P400 by Participant')
plt.grid(True)
plt.tight_layout()  # Adjust the layout to make room for the legend
plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\1617307158.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed


In [72]:
"""Value differences"""

# Create a color map for participants
participants = df2122['Participant'].unique()  # Extract unique participants
color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed

# Create a dictionary to map each participant to a color
participant_colors = {participant: color_map(i) for i, participant in enumerate(participants)}

# Initializing lists to store amplitude differences
study_state_amplitude_diff = []
random_amplitude_diff = []

# Calculating differences
for participant in participants:
    # Filter by participant
    unamb_study_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Steady State')]['P400_Latency'].mean()
    amb_study_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Steady State')]['P400_Latency'].mean()
    unamb_random_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Unambiguous') & (df2122['Randomness'] == 'Random')]['P400_Latency'].mean()
    amb_random_amplitude = df2122[(df2122['Participant'] == participant) & (df2122['Ambiguity'] == 'Ambiguous') & (df2122['Randomness'] == 'Random')]['P400_Latency'].mean()
    
    # Calculate differences and append to lists
    study_state_amplitude_diff.append(unamb_study_amplitude - amb_study_amplitude)
    random_amplitude_diff.append(unamb_random_amplitude - amb_random_amplitude)

# Plotting
plt.figure(figsize=(10, 10))
for i, participant in enumerate(participants):
    plt.scatter(study_state_amplitude_diff[i], random_amplitude_diff[i], color=participant_colors[participant], label=f' {participant}')
# make axis same range
buffer = 0.0018
xlim_min = min(study_state_amplitude_diff) - buffer
xlim_max = max(study_state_amplitude_diff)
ylim_min = min(random_amplitude_diff)
ylim_max = max(random_amplitude_diff) 

# Ensure the plot is square by setting the same limits for both axes
combined_min = min(xlim_min, ylim_min)
combined_max = max(xlim_max, ylim_max)
plt.plot([combined_min, combined_max], [combined_min, combined_max], color='brown', linestyle='--')
plt.xlim(combined_min, combined_max)
plt.ylim(combined_min, combined_max)

# Enhance the legend to show unique entries only
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0.)

plt.xlabel('Steady State Latency Difference (Unambiguous - Ambiguous)')
plt.ylabel('Random State Latency Difference (Unambiguous - Ambiguous)')
plt.title('Scatter Plot of Latency Differences at P400 by Participant')
plt.grid(True)
plt.tight_layout()  # Adjust the layout to make room for the legend
plt.show()


C:\Users\mtu10\AppData\Local\Temp\ipykernel_9076\3495064017.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_map = plt.cm.get_cmap('tab20', len(participants))  # 'tab20' has 20 distinct colors, adjust if needed


In [73]:
"""Calculate group mean, median, and standart deviation"""

# Define the ages in each group
group1 = np.array([28, 25, 26, 24, 32])
group2 = np.array([24, 25, 27, 26, 22, 25, 27, 26, 23])
group3 = np.array([26, 25, 21, 22, 23, 28, 24, 22, 23, 23, 29])
group4 = np.array([27, 23, 24, 27])

# Define a function to calculate mean, median, and standard deviation
def calculate_stats(data):
    mean = np.mean(data)
    median = np.median(data)
    std_dev = np.std(data, ddof=1)  # Using sample standard deviation (N-1)
    return mean, median, std_dev

# Calculate stats for each group
stats_group1 = calculate_stats(group1)
stats_group2 = calculate_stats(group2)
stats_group3 = calculate_stats(group3)
stats_group4 = calculate_stats(group4)

# Combine all groups into one for overall statistics
all_groups = np.concatenate((group1, group2, group3, group4))
stats_all_groups = calculate_stats(all_groups)
stats_group1, stats_group2, stats_group3, stats_group4, stats_all_groups

((27.0, 26.0, 3.1622776601683795),
 (25.0, 25.0, 1.7320508075688772),
 (24.181818181818183, 23.0, 2.5619594773603196),
 (25.25, 25.5, 2.0615528128088303),
 (25.06896551724138, 25.0, 2.4630270934470424))